*This study was conducted for skills demonstration purposes only*

# **Forecasting the UK Construction Sector with Macroeconomic Indicators**
# Section 3. Data Preprocessing

This section pre-processes the collected in the '2_Data Collection.ipynb' file time series to make them suitable for analysis and modeling. 

Key tasks include:
1. Cleaning metadata and empty rows/columns
2. Cleaning out irrelevant data beyond the period from January 2005 to January 2025
3. converting annual and quarterly data to monthly format
4. Filling in missing data
6. Merging all data sets into a single frame
7. Normalizing and standardizing numerical variables
8. Detecting and handling outliers


### 1. Collected Data

In 2_Data_Collection.ipynb file time series data on macroeconomic and construction sector indicators from official statistical sources were collected. The list of saved datasets is presented in the table below.


| N  | **Indicators**                                        | Frequency                        | Data Source                           | File Name                                         |
|----|------------------------------------------------------|---------------------------------|-------------------------------------|--------------------------------------------------|
|    | **Macroeconomic Indicators (independent variables)**|                                 |                                     |                                                  |
| 1  | Consumer Price Index incl. housing (CPIH) / Inflation| annually, quarterly, monthly     | Office for National Statistics (ONS)| df_1_raw.csv                                     |
| 2  | GDP growth rate (UK, real)                           | annually, quarterly             | Office for National Statistics (ONS)| df_2_raw.csv                                     |
| 3  | Employment rate or unemployment rate                 | annually, quarterly, monthly    | Office for National Statistics (ONS)| df_3_raw.csv                                     |
| 4  | Interest rate (Bank of England base rate)            | monthly                        | Bank of England                     | df_4_raw.csv                                     |
| 5  | Exchange rate (GBP/USD and GBP/EUR)                   | monthly                        | Bank of England                     | df_5_1_raw.csv, df_5_2_raw.csv                   |
| 6  | Business investment (gross fixed capital formation)  | quarterly                     | Office for National Statistics (ONS)| df_6_raw.csv                                     |
| 7  | Government spending                                   | annually, quarterly, monthly    | Office for National Statistics (ONS)| df_7_raw.csv                                     |
|    |                                                      |                                 |                                     |                                                  |
|    | **Construction Sector Indicators (dependent variables)**|                               |                                     |                                                  |
| 8  | Construction output (total, residential, commercial) | annually, quarterly, monthly     | Office for National Statistics (ONS)| df_8_raw.csv                                     |
| 9  | Construction material prices                          | monthly                        | Department for Business & Trade Gov.uk | df_9_1_raw.csv, df_9_2_raw.csv, df_9_3_raw.csv, df_9_4_raw.csv |
| 10 | Number of construction firms                          | annually                      | Office for National Statistics (ONS)| df_10_raw.csv                                    |
| 11 | Number of employees in the construction sector       | annually                      | Office for National Statistics (ONS)| df_11_raw.csv                                    |
| 12 | Number and value of new construction contracts/orders| annually, quarterly           | Office for National Statistics (ONS)| df_12_raw.csv                                    |


### 2. Tools and Libraries

In [1]:
pip install odfpy

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install ezodf odfpy

Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install xlrd

Note: you may need to restart the kernel to use updated packages.


In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import requests
from io import BytesIO
from io import StringIO
import ezodf

### 3. Functions for data preprocessing

In [5]:
#Function for Changing type of 'Date' Column from str (yyyy Q) for datetime64[ns]

def str_yyyy_Q_to_datetime(df, date_column):
    #As an assumption, we will assign the average value for the quarter to the first day of the quarter
    
    #Split 'Date' into year and quarter number
    df[['Year', 'Quarter']] = df[date_column].str.extract(r'(\d{4})\s+Q([1-4])')

    #Convert to integers
    df['Year'] = df['Year'].astype('int')
    df['Quarter'] = df['Quarter'].astype('int')

    #Map quarter to starting month
    df['Month'] = df['Quarter'].map({1:1, 2:4, 3:7, 4:10}) 

    #Map quarter to starting month
    df[date_column] = pd.to_datetime(dict(year=df['Year'], month=df['Month'], day=1)) 

    # Drop helper columns
    df = df.drop(columns =['Year', 'Quarter', 'Month'])

    return df

In [6]:
#Function for Changing type of 'Date' Column from str (yyyy-q) for datetime64[ns]

def str_yyyy___q_to_datetime(df, date_column):
    #As an assumption, we will assign the average value for the quarter to the first day of the quarter
    
    #Split 'Date' into year and quarter number
    df[['Year', 'Quarter']] = df[date_column].str.extract(r'(?i)(\d{4})[\s\-]*q([1-4])')

     # Drop rows where extraction failed
    df = df.dropna(subset=['Year', 'Quarter'])

    #Convert to integers
    df['Year'] = df['Year'].astype('int')
    df['Quarter'] = df['Quarter'].astype('int')

    #Map quarter to starting month
    df['Month'] = df['Quarter'].map({1:1, 2:4, 3:7, 4:10}) 

    #Map quarter to starting month
    df[date_column] = pd.to_datetime(dict(year=df['Year'], month=df['Month'], day=1)) 

    # Drop helper columns
    df = df.drop(columns =['Year', 'Quarter', 'Month'])

    return df

In [7]:
# Function for Converting Quarterly Data to Monthly
def quarter_to_month_freq(df, date_column, variable_column):
    df = df.set_index(date_column).resample('MS').ffill().reset_index()
    df[variable_column] = df[variable_column].astype('float') / 3
    df[variable_column] = df[variable_column].round(2)
    return df

In [8]:
# Function for Converting Quarterly Data to Monthly in multiple columns
def quarter_to_month_freq_for_list_of_columns(df, date_column, variable_columns):
    df = df.set_index(date_column).resample('MS').ffill().reset_index()
    for col in variable_columns:
        df[col] = df[col].astype('float') / 3
        df[col] = df[col].round(2)
    return df

In [9]:
#Function for reshaping data from wide format to long date format:
def reshape_from_wide_to_date_series(df, value_name): 

    #Melt the DataFrame to long format
    df_new = df.melt(id_vars=df.columns[0], 
                          var_name='Year', 
                          value_name=value_name)

    #Rename the month column
    df_new = df_new.rename(columns={df.columns[0]: 'Month'})

    # Combine 'Year' and 'Month' into a 'Date' column
    df_new['Date'] = pd.to_datetime(df_new['Year'].astype(str) + '-' + df_new['Month'].str.strip(), format='%Y-%B')

    # Drop 'Year' and 'Month' if not needed
    df_new = df_new[[value_name, 'Date']]

    # Sort by date
    df_new = df_new.sort_values('Date').reset_index(drop=True)

    return df_new

In [10]:
# Primary data cleaning from meta-data and empty rows and columns (for .xls and .ods files)
def cleaning_meta_data_empty_rows_columns(df_raw, N_extra_upper_rows):

    # Drop upper rows 
    df = df_raw[N_extra_upper_rows:]

    #Set cleaned row 0 as the header
    new_header = []
    for col in df.iloc[0]:
        if pd.isna(col):
            new_header.append('')
        elif isinstance(col, float):
            new_header.append(int(col))
        else:
            new_header.append(str(col))
    
    df = df[1:]                      # take the data less the header row
    df.columns = new_header  

    # Drop completely empty rows and columns
    df = df.dropna(axis=0, how='all')
    df = df.dropna(axis=1, how='all')

    # Reset the index
    df.reset_index(drop=True, inplace=True)

    return df

#### Indicator 1: Consumer Price Index incl. housing (CPIH)

- Source: Office for National Statistics (ONS)
- Frequency: annually, quarterly, monthly
- Coverage: Jan 1988 - May 2025
- CPIH INDEX 00: ALL ITEMS 2015=100
- Release date: 18-Jun-2025
- Next release: 16-Jul-2025

In [11]:
#Extract data from a saved file
df_1 = pd.read_csv('df_1_raw.csv')
df_1.head(5)

,Date,CPIH
0,1988,48.2
1,1989,51.0
2,1990,55.1
3,1991,59.2
4,1992,61.9


In [12]:
#Check info of the dataframe
df_1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 637 entries, 0 to 636
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Date    637 non-null    object 
 1   CPIH    637 non-null    float64
dtypes: float64(1), object(1)
memory usage: 10.1+ KB


In [13]:
# Determine the required data interval:
print(df_1[df_1['Date'] == '2005 JAN'])
print(df_1[df_1['Date'] == '2025 JAN'])

         Date  CPIH
391  2005 JAN  78.3
         Date   CPIH
631  2025 JAN  135.1


In [14]:
#Extracting the required data interval:
df_1 = df_1.iloc[391:632].reset_index(drop=True)
df_1

,Date,CPIH
0,2005 JAN,78.3
1,2005 FEB,78.5
2,2005 MAR,78.8
3,2005 APR,79.1
4,2005 MAY,79.4
...,...,...
236,2024 SEP,133.5
237,2024 OCT,134.3
238,2024 NOV,134.6
239,2024 DEC,135.1


In [15]:
#Changing the 'Date' column type to datetime64[ns]
#For convenience, we will assign the average value for the month to the first day of the month
df_1['Date'] = pd.to_datetime(df_1['Date'], format='%Y %b')

#Setting the 'Date' column as an index column
df_1 = df_1.set_index('Date')

print(df_1.dtypes)
df_1.head(5)

CPIH    float64
dtype: object


,CPIH
Date,
2005-01-01,78.3
2005-02-01,78.5
2005-03-01,78.8
2005-04-01,79.1
2005-05-01,79.4


#### Indicator 2: Gross Domestic Product (GDP). Chained volume measures: Seasonally adjusted £m

- Source: Office for National Statistics (ONS)
- Frequency: annually, quarterly
- Coverage: Q1 1955 - Q1 2025
- Release date: 15-May-2025
- Next release: 30-Jun-2025

In [16]:
#Extract data from a saved file
df_2 = pd.read_csv('df_2_raw.csv')
df_2.head(5)

,Date,"GDP, £m"
0,1948,422621
1,1949,436620
2,1950,451212
3,1951,467977
4,1952,474994


In [17]:
#Check info of the dataframe
df_2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 358 entries, 0 to 357
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   Date     358 non-null    object
 1   GDP, £m  358 non-null    int64 
dtypes: int64(1), object(1)
memory usage: 5.7+ KB


In [18]:
# Determine the required data interval:
print(df_2[df_2['Date'] == '2005 Q1'])
print(df_2[df_2['Date'] == '2025 Q1'])

        Date  GDP, £m
277  2005 Q1   502702
        Date  GDP, £m
357  2025 Q1   646833


In [19]:
#Extracting the required data interval:
df_2 = df_2.iloc[277:].reset_index(drop=True)
df_2

,Date,"GDP, £m"
0,2005 Q1,502702
1,2005 Q2,506906
2,2005 Q3,511132
3,2005 Q4,515935
4,2006 Q1,518495
...,...,...
76,2024 Q1,638746
77,2024 Q2,641670
78,2024 Q3,641675
79,2024 Q4,642287


In [20]:
#Changing the 'Date' column type to datetime64[ns]
#For convenience, we will assign the average value for the month to the first day of the month
df_2 = str_yyyy_Q_to_datetime(df_2, 'Date')

# Converting quarterly frequency to monthly
df_2 = quarter_to_month_freq(df_2, 'Date', 'GDP, £m')

#Setting the 'Date' column as an index column
df_2 = df_2.set_index('Date')

print(df_2.dtypes)
df_2.head(5)

GDP, £m    float64
dtype: object


,"GDP, £m"
Date,
2005-01-01,167567.33
2005-02-01,167567.33
2005-03-01,167567.33
2005-04-01,168968.67
2005-05-01,168968.67


#### Indicator 3: Employment rate (aged 16 to 64, seasonally adjusted): %
- Source: Office for National Statistics (ONS)
- Frequency: annually, quarterly, monthly
- Coverage: Feb 1971 - Mar 2025
- Release date: 10-Jun-2025
- Next release: 17-Jul-2025

In [21]:
#Extract data from a saved file
df_3 = pd.read_csv('df_3_raw.csv')
df_3.head(5)

,Date,"Employment rate, %"
0,1971,71.8
1,1972,72.0
2,1973,72.9
3,1974,73.0
4,1975,72.6


In [22]:
#Check info of the dataframe
df_3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 922 entries, 0 to 921
Data columns (total 2 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Date                922 non-null    object 
 1   Employment rate, %  922 non-null    float64
dtypes: float64(1), object(1)
memory usage: 14.5+ KB


In [23]:
# Determine the required data interval:
print(df_3[df_3['Date'] == '2005 JAN'])
print(df_3[df_3['Date'] == '2025 JAN'])

         Date  Employment rate, %
678  2005 JAN                73.2
         Date  Employment rate, %
918  2025 JAN                75.1


In [24]:
#Extracting the required data interval:
df_3 = df_3.iloc[678:919].reset_index(drop=True)
df_3

,Date,"Employment rate, %"
0,2005 JAN,73.2
1,2005 FEB,73.1
2,2005 MAR,72.9
3,2005 APR,72.9
4,2005 MAY,72.9
...,...,...
236,2024 SEP,74.9
237,2024 OCT,74.9
238,2024 NOV,75.0
239,2024 DEC,75.0


In [25]:
#Changing the 'Date' column type to datetime64[ns]
#For convenience, we will assign the average value for the month to the first day of the month
df_3['Date'] = pd.to_datetime(df_3['Date'], format='%Y %b')

#Setting the 'Date' column as an index column
df_3 = df_3.set_index('Date')

print(df_3.dtypes)
df_3.head(5)

Employment rate, %    float64
dtype: object


,"Employment rate, %"
Date,
2005-01-01,73.2
2005-02-01,73.1
2005-03-01,72.9
2005-04-01,72.9
2005-05-01,72.9


#### Indicator 4: Month average Bank Rate

- Source: Bank of England (BoE)
- Frequency: monthly
- Coverage: Jan 2000 - Jun 2025
- Release date: 30-Jun-2025
- Next release: 31-Jul-2025

In [26]:
#Extract data from a saved file
df_4 = pd.read_csv('df_4_raw.csv')
df_4.head(5)

,Date,"BoE Rate, %"
0,2005-01-31,4.75
1,2005-02-28,4.75
2,2005-03-31,4.75
3,2005-04-30,4.75
4,2005-05-31,4.75


In [27]:
#Check info of the dataframe
df_4.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 241 entries, 0 to 240
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Date         241 non-null    object 
 1   BoE Rate, %  241 non-null    float64
dtypes: float64(1), object(1)
memory usage: 3.9+ KB


In [28]:
#Changing the 'Date' column type to datetime64[ns]
df_4['Date'] = pd.to_datetime(df_4['Date'])

#For convenience, we will reassign the average value for the month to the first day of the month
df_4['Date'] = df_4['Date'].apply(lambda d: d.replace(day=1))

df_4.head(5)

,Date,"BoE Rate, %"
0,2005-01-01,4.75
1,2005-02-01,4.75
2,2005-03-01,4.75
3,2005-04-01,4.75
4,2005-05-01,4.75


In [29]:
# Determine the required data interval:
print(df_4[df_4['Date'] == '2005-01-01'])
print(df_4[df_4['Date'] == '2025-01-01'])

        Date  BoE Rate, %
0 2005-01-01         4.75
          Date  BoE Rate, %
240 2025-01-01         4.75


In [30]:
#Extracting the required data interval:
df_4 = df_4.iloc[:241].reset_index(drop=True)

#Setting the 'Date' column as an index column
df_4 = df_4.set_index('Date')
df_4

,"BoE Rate, %"
Date,
2005-01-01,4.7500
2005-02-01,4.7500
2005-03-01,4.7500
2005-04-01,4.7500
2005-05-01,4.7500
...,...
2024-09-01,5.0000
2024-10-01,5.0000
2024-11-01,4.7976


#### Indicators 5.1 and 5.2: Exchange rates (GBP/USD and GBP/EUR)
- Source: Bank of England (BoE)
- Frequency: monthly average
- Coverage: Jan 2005 - Jul 2025
- Link update : 15-Jul-2025
- Release: every day

**GBP/EUR**

In [31]:
#Extract data from a saved file
df_5_1 = pd.read_csv('df_5_1_raw.csv')
df_5_1.head(5)

,Date,GBP/EUR
0,2005-01-31,1.4331
1,2005-02-28,1.4499
2,2005-03-31,1.4440
3,2005-04-30,1.4652
4,2005-05-31,1.4611


In [32]:
#Check info of the dataframe
df_5_1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 246 entries, 0 to 245
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   Date     246 non-null    object 
 1   GBP/EUR  246 non-null    float64
dtypes: float64(1), object(1)
memory usage: 4.0+ KB


In [33]:
#Changing the 'Date' column type to datetime64[ns]
df_5_1['Date'] = pd.to_datetime(df_5_1['Date'])

#For convenience, we will reassign the average value for the month to the first day of the month
df_5_1['Date'] = df_5_1['Date'].apply(lambda d: d.replace(day=1))

df_5_1.head(5)

,Date,GBP/EUR
0,2005-01-01,1.4331
1,2005-02-01,1.4499
2,2005-03-01,1.4440
3,2005-04-01,1.4652
4,2005-05-01,1.4611


In [34]:
# Determine the required data interval:
print(df_5_1[df_5_1['Date'] == '2005-01-01'])
print(df_5_1[df_5_1['Date'] == '2025-01-01'])

        Date  GBP/EUR
0 2005-01-01   1.4331
          Date  GBP/EUR
240 2025-01-01   1.1921


In [35]:
#Extracting the required data interval:
df_5_1 = df_5_1.iloc[:241].reset_index(drop=True)

#Setting the 'Date' column as an index column
df_5_1 = df_5_1.set_index('Date')
df_5_1

,GBP/EUR
Date,
2005-01-01,1.4331
2005-02-01,1.4499
2005-03-01,1.4440
2005-04-01,1.4652
2005-05-01,1.4611
...,...
2024-09-01,1.1903
2024-10-01,1.1972
2024-11-01,1.1998


**GBP/USD**

In [36]:
#Extract data from a saved file
df_5_2 = pd.read_csv('df_5_2_raw.csv')
df_5_2.head(5)

,Date,GBP/USD
0,2005-01-31,1.8764
1,2005-02-28,1.8871
2,2005-03-31,1.9078
3,2005-04-30,1.8960
4,2005-05-31,1.8538


In [37]:
#Check info of the dataframe
df_5_2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 247 entries, 0 to 246
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   Date     247 non-null    object 
 1   GBP/USD  247 non-null    float64
dtypes: float64(1), object(1)
memory usage: 4.0+ KB


In [38]:
#Changing the 'Date' column type to datetime64[ns]
df_5_2['Date'] = pd.to_datetime(df_5_2['Date'])

#For convenience, we will reassign the average value for the month to the first day of the month
df_5_2['Date'] = df_5_2['Date'].apply(lambda d: d.replace(day=1))
df_5_2.head(5)

,Date,GBP/USD
0,2005-01-01,1.8764
1,2005-02-01,1.8871
2,2005-03-01,1.9078
3,2005-04-01,1.8960
4,2005-05-01,1.8538


In [39]:
# Determine the required data interval:
print(df_5_2[df_5_2['Date'] == '2005-01-01'])
print(df_5_2[df_5_2['Date'] == '2025-01-01'])

        Date  GBP/USD
0 2005-01-01   1.8764
          Date  GBP/USD
240 2025-01-01   1.2348


In [40]:
#Extracting the required data interval:
df_5_2 = df_5_2.iloc[:241].reset_index(drop=True)

#Setting the 'Date' column as an index column
df_5_2 = df_5_2.set_index('Date')
df_5_2

,GBP/USD
Date,
2005-01-01,1.8764
2005-02-01,1.8871
2005-03-01,1.9078
2005-04-01,1.8960
2005-05-01,1.8538
...,...
2024-09-01,1.3217
2024-10-01,1.3045
2024-11-01,1.2750


In [41]:
df_5 = pd.merge(df_5_1, df_5_2, on='Date', how='inner')
df_5.head(5)

,GBP/EUR,GBP/USD
Date,,
2005-01-01,1.4331,1.8764
2005-02-01,1.4499,1.8871
2005-03-01,1.4440,1.9078
2005-04-01,1.4652,1.8960
2005-05-01,1.4611,1.8538


#### Indicator 6: Business Investment (CVM, Seasonally Adjusted, in £ millions)
- Source: Office for National Statistics (ONS)
- Frequency: quarterly
- Coverage: Q2 1997 - Q1 2025
- Release date: 15-May-2025
- Next release: 30-Jun-2025

In [42]:
#Extract data from a saved file
df_6 = pd.read_csv('df_6_raw.csv')
df_6.head(5)

,Date,"Business Investment, % change"
0,1997 Q2,4.4
1,1997 Q3,6.1
2,1997 Q4,2.6
3,1998 Q1,1.6
4,1998 Q2,-0.4


In [43]:
#Check info of the dataframe
df_6.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 112 entries, 0 to 111
Data columns (total 2 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   Date                           112 non-null    object 
 1   Business Investment, % change  112 non-null    float64
dtypes: float64(1), object(1)
memory usage: 1.9+ KB


In [44]:
# Determine the required data interval:
print(df_6[df_6['Date'] == '2005 Q1'])
print(df_6[df_6['Date'] == '2025 Q1'])

       Date  Business Investment, % change
31  2005 Q1                           -0.5
        Date  Business Investment, % change
111  2025 Q1                            5.9


In [45]:
#Extracting the required data interval:
df_6 = df_6.iloc[31:].reset_index(drop=True)
df_6

,Date,"Business Investment, % change"
0,2005 Q1,-0.5
1,2005 Q2,55.6
2,2005 Q3,-32.2
3,2005 Q4,-0.5
4,2006 Q1,5.1
...,...,...
76,2024 Q1,-0.2
77,2024 Q2,1.8
78,2024 Q3,2.2
79,2024 Q4,-1.9


In [46]:
#Changing the 'Date' column type to datetime64[ns]
#For convenience, we will assign the average value for the month to the first day of the month
df_6 = str_yyyy_Q_to_datetime(df_6, 'Date')

# Converting quarterly frequency to monthly
df_6 = quarter_to_month_freq(df_6, 'Date', 'Business Investment, % change')

#Setting the 'Date' column as an index column
df_6 = df_6.set_index('Date')

print(df_6.dtypes)
df_6.head(5)

Business Investment, % change    float64
dtype: object


,"Business Investment, % change"
Date,
2005-01-01,-0.17
2005-02-01,-0.17
2005-03-01,-0.17
2005-04-01,18.53
2005-05-01,18.53


#### Indicator 7: Total managed expenditure: £m

- Source: Office for National Statistics (ONS)
- Frequency: annually, quarterly, monthly
- Coverage: Apr 1997 - May 2025
- Release date: 20-Jun-2025
- Next release: 20-Jul-2025

In [47]:
#Extract data from a saved file
df_7 = pd.read_csv('df_7_raw.csv')
df_7.head(5)

,Date,"Govt Expenditure, £m"
0,1946,4353
1,1947,3925
2,1948,4327
3,1949,4655
4,1950,4768


In [48]:
#Check info of the dataframe
df_7.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 736 entries, 0 to 735
Data columns (total 2 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   Date                  736 non-null    object
 1   Govt Expenditure, £m  736 non-null    int64 
dtypes: int64(1), object(1)
memory usage: 11.6+ KB


In [49]:
# Determine the required data interval:
print(df_7[df_7['Date'] == '2005 JAN'])
print(df_7[df_7['Date'] == '2025 JAN'])

         Date  Govt Expenditure, £m
490  2005 JAN                 48114
         Date  Govt Expenditure, £m
730  2025 JAN                111482


In [50]:
#Extracting the required data interval:
df_7 = df_7.iloc[490:731].reset_index(drop=True)
df_7

,Date,"Govt Expenditure, £m"
0,2005 JAN,48114
1,2005 FEB,46543
2,2005 MAR,46066
3,2005 APR,49028
4,2005 MAY,42293
...,...,...
236,2024 SEP,106042
237,2024 OCT,108156
238,2024 NOV,102631
239,2024 DEC,113354


In [51]:
#Changing the 'Date' column type to datetime64[ns]
#For convenience, we will assign the average value for the month to the first day of the month
df_7['Date'] = pd.to_datetime(df_7['Date'], format='%Y %b')

#Setting the 'Date' column as an index column
df_7 = df_7.set_index('Date')

print(df_7.dtypes)
df_7.head(5)

Govt Expenditure, £m    int64
dtype: object


,"Govt Expenditure, £m"
Date,
2005-01-01,48114
2005-02-01,46543
2005-03-01,46066
2005-04-01,49028
2005-05-01,42293


#### Indicator 8: Construction output (Seasonally Adjusted, total, residential, commercial): £m
- Source: Office for National Statistics (ONS)
- Frequency: annually, quarterly, monthly (from 2010)
- Coverage: Q1 1997 - May 2025
- Release date: 11-Jul-2025
- Next release: 14-Aug-2025

In [52]:
#Extract data from a saved file
df_8 = pd.read_csv('df_8_raw.csv')
df_8.head(5)

,v4_1,Data Marking,years-quarters-months,Time,administrative-geography,Geography,seasonal-adjustment,SeasonalAdjustment,construction-series-type,SeriesType,construction-classifications,TypeOfWork
0,1570.0,NaN,2013-aug,2013 - Aug,K03000001,Great Britain,seasonal-adjustment,Seasonally adjusted,pounds-million,£million,1-2-3,Infrastructure new work
1,2588.0,NaN,2013-aug,2013 - Aug,K03000001,Great Britain,seasonal-adjustment,Seasonally adjusted,pounds-million,£million,1-2-2-1-2-2,Private commercial new work
2,393.0,NaN,2013-aug,2013 - Aug,K03000001,Great Britain,seasonal-adjustment,Seasonally adjusted,pounds-million,£million,1-2-2-1-2-1,Private industrial new work
3,13367.0,NaN,2013-aug,2013 - Aug,K03000001,Great Britain,seasonal-adjustment,Seasonally adjusted,pounds-million,£million,1,All work
4,2593.0,NaN,2013-aug,2013 - Aug,K03000001,Great Britain,seasonal-adjustment,Seasonally adjusted,pounds-million,£million,1-1-1,Total housing R&M


In [53]:
#Check info of the dataframe
df_8.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4564 entries, 0 to 4563
Data columns (total 12 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   v4_1                          4564 non-null   float64
 1   Data Marking                  0 non-null      float64
 2   years-quarters-months         4564 non-null   object 
 3   Time                          4564 non-null   object 
 4   administrative-geography      4564 non-null   object 
 5   Geography                     4564 non-null   object 
 6   seasonal-adjustment           4564 non-null   object 
 7   SeasonalAdjustment            4564 non-null   object 
 8   construction-series-type      4564 non-null   object 
 9   SeriesType                    4564 non-null   object 
 10  construction-classifications  4564 non-null   object 
 11  TypeOfWork                    4564 non-null   object 
dtypes: float64(2), object(10)
memory usage: 428.0+ KB


In [54]:
#Filtering data by the column "TypeOfWork" for "All Works"
df_8 = df_8[df_8['TypeOfWork']=='All work']

#Removing unnecessary columns
df_8.drop(columns = ['Data Marking', 'Time', 'administrative-geography', 'Geography', 'seasonal-adjustment', 'SeasonalAdjustment', 'construction-series-type', 'SeriesType', 'construction-classifications', 'TypeOfWork'])  

#Swap 'v4_1' and 'years-quarters-months'
df_8 = df_8[['years-quarters-months', 'v4_1']]

# Changing the header
df_8.columns = ['Date', 'Construction output, £m']

df_8.head(5)

,Date,"Construction output, £m"
3,2013-aug,13367.0
16,2016-jul,15467.0
32,2018-q2,48955.0
45,2019-jul,16757.0
68,2007-q1,45038.0


In [55]:
# Filtering Data by column 'years-quarters-months' for only data in monthly expression and quarterly data from 2005 till 2010 years
df_8_1 = df_8[df_8['Date'].str.match(r'^\d{4}-[a-z]{3}$', case=False)]
df_8_2 = df_8[df_8['Date'].str.match(r'^20(0[5-9]|10)-q[1-4]$', case=False)]

df_8_1 = df_8_1.reset_index(drop=True)
df_8_2 = df_8_2.reset_index(drop=True)

print(df_8_2.head(5))
print(df_8_1.head(5))

      Date  Construction output, £m
0  2007-q1                  45038.0
1  2005-q1                  44098.0
2  2005-q2                  43778.0
3  2009-q2                  37434.0
4  2010-q1                  39437.0
       Date  Construction output, £m
0  2013-aug                  13367.0
1  2016-jul                  15467.0
2  2019-jul                  16757.0
3  2011-aug                  13577.0
4  2016-nov                  15557.0


In [56]:
#Changing the 'Date' column type to datetime64[ns]
#For convenience, we will assign the average value for the month to the first day of the month
df_8_1['Date'] = pd.to_datetime(df_8_1['Date'], format='%Y-%b')
df_8_2 = str_yyyy___q_to_datetime(df_8_2, 'Date')

#Sort by 'Date'
df_8_1 = df_8_1.sort_values(by='Date')
df_8_2 = df_8_2.sort_values(by='Date')
df_8_2 = quarter_to_month_freq(df_8_2, 'Date', 'Construction output, £m')

df_8_1 = df_8_1.reset_index(drop=True)
df_8_2 = df_8_2.reset_index(drop=True)

print(df_8_2.tail(5))
print(df_8_1.head(5))

         Date  Construction output, £m
65 2010-06-01                 13791.33
66 2010-07-01                 14045.33
67 2010-08-01                 14045.33
68 2010-09-01                 14045.33
69 2010-10-01                 13672.00
        Date  Construction output, £m
0 2010-01-01                  12750.0
1 2010-02-01                  13311.0
2 2010-03-01                  13377.0
3 2010-04-01                  13559.0
4 2010-05-01                  13830.0


In [57]:
#Determine the required data intervals:
print(df_8_2[df_8_2['Date'] == '2005-01-01'])
print(df_8_2[df_8_2['Date'] == '2009-12-01'])
print(df_8_1[df_8_1['Date'] == '2010-01-01'])
print(df_8_1[df_8_1['Date'] == '2025-01-01'])

        Date  Construction output, £m
0 2005-01-01                 14699.33
         Date  Construction output, £m
59 2009-12-01                  12545.0
        Date  Construction output, £m
0 2010-01-01                  12750.0
          Date  Construction output, £m
180 2025-01-01                  17700.0


In [58]:
#Extracting the required data interval:
df_8_2 = df_8_2.iloc[:60]
df_8_1 = df_8_1.iloc[:181]

df_8_2.tail(5)

,Date,"Construction output, £m"
55,2009-08-01,12700.0
56,2009-09-01,12700.0
57,2009-10-01,12545.0
58,2009-11-01,12545.0
59,2009-12-01,12545.0


In [59]:
# Merging df_8_1 and df_8_2
df_8 = pd.concat([df_8_2, df_8_1], ignore_index=True)

#Setting the 'Date' column as an index column
df_8 = df_8.set_index('Date')

df_8

,"Construction output, £m"
Date,
2005-01-01,14699.33
2005-02-01,14699.33
2005-03-01,14699.33
2005-04-01,14592.67
2005-05-01,14592.67
...,...
2024-09-01,17692.00
2024-10-01,17684.00
2024-11-01,17795.00


#### Indicator 9: Construction material Price Indices

**9_1. Construction Material price indices for 2020 - 2025 years**

- Source: Department for Business & Trade Gov.uk
- Frequency: monthly
- Coverage: Jan 2020 - Jan 2025
- Release date: 02-Jul-2025
- Next release: 06-Aug-2025
- Base Index: 2015 = 100

In [60]:
#Extract data from a saved file
df_9_1 = pd.read_csv('df_9_1_raw.csv')
df_9_1.head(10)

,0,1,2,3,4,5
0,Table 1a: Monthly construction material price ...,NaN,NaN,NaN,NaN,NaN
1,The publication of data contained in this work...,NaN,NaN,NaN,NaN,NaN
2,This worksheet contains one table. Some cells ...,NaN,NaN,NaN,NaN,NaN
3,"Freeze panes are turned on. To turn off, selec...",NaN,NaN,NaN,NaN,NaN
4,Source: DBT/ONS/Building cost information serv...,NaN,NaN,NaN,NaN,NaN
5,Base year: 2015=100,NaN,NaN,NaN,NaN,NaN
6,Year,Month,New Housing,Other New Work,Repair and Maintenance,All Work
7,2020,January,110.9,110.6,111.3,110.7
8,2020,February,111.4,110.7,111.7,111
9,2020,March,111.7,111.1,111.7,111.2


In [61]:
#Data cleaning from meta-data and empty rows and columns
df_9_1 = cleaning_meta_data_empty_rows_columns(df_9_1, 6)
df_9_1.head(3)

,Year,Month,New Housing,Other New Work,Repair and Maintenance,All Work
0,2020,January,110.9,110.6,111.3,110.7
1,2020,February,111.4,110.7,111.7,111
2,2020,March,111.7,111.1,111.7,111.2


In [62]:
#Check info of the dataframe
df_9_1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 61 entries, 0 to 60
Data columns (total 6 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   Year                    61 non-null     object
 1   Month                   61 non-null     object
 2   New Housing             61 non-null     object
 3   Other New Work          61 non-null     object
 4   Repair and Maintenance  61 non-null     object
 5   All Work                61 non-null     object
dtypes: object(6)
memory usage: 3.0+ KB


In [63]:
#Creating the 'Date' column of type datetime64[ns]
#For convenience, we will assign the average value for the month to the first day of the month
#Convert to integers
df_9_1['Year'] = df_9_1['Year'].astype('int')

#Map month to start in JAN, FEB, MAR format
df_9_1['Month_formated'] = df_9_1['Month'].map({'January':1, 'February':2, 'March':3, 'April':4, 'May':5, 'June':6, 'July':7, 'August':8, 'September':9, 'October':10, 'November':11, 'December':12, 'November [p]':11, 'December [p]':12, 'January [p]':1}) 

#Create a column 'Date' in datetime64 format
df_9_1['Date'] = pd.to_datetime(dict(year=df_9_1['Year'], month=df_9_1['Month_formated'], day=1)) 

print(df_9_1.dtypes)
df_9_1.head(5)

Year                               int64
Month                             object
New Housing                       object
Other New Work                    object
Repair and Maintenance            object
All Work                          object
Month_formated                     int64
Date                      datetime64[ns]
dtype: object


,Year,Month,New Housing,Other New Work,Repair and Maintenance,All Work,Month_formated,Date
0,2020,January,110.9,110.6,111.3,110.7,1,2020-01-01
1,2020,February,111.4,110.7,111.7,111,2,2020-02-01
2,2020,March,111.7,111.1,111.7,111.2,3,2020-03-01
3,2020,April,111.7,112.4,111.7,113.5,4,2020-04-01
4,2020,May,112.5,112.8,112.2,113.9,5,2020-05-01


In [64]:
#Drop unnecessary columns
df_9_1 = df_9_1.drop(columns = ['Year', 'Month', 'Month_formated','New Housing', 'Other New Work', 'Repair and Maintenance'])

#Rename columns
df_9_1.columns = ['Construction Material Price Index, 2015 = 100', 'Date']

df_9_1.head(5)

,"Construction Material Price Index, 2015 = 100",Date
0,110.7,2020-01-01
1,111,2020-02-01
2,111.2,2020-03-01
3,113.5,2020-04-01
4,113.9,2020-05-01


**9_2. Construction Material price indices for 2015 - 2020 years**

- Source: Department for Business & Trade Gov.uk
- Frequency: monthly
- Coverage: Jan 2015 - Dec 2020
- Release date: 03-Feb-2021
- Base Index: 2015 = 100

In [65]:
#Extract data from a saved file
df_9_2 = pd.read_csv('df_9_2_raw.csv')
df_9_2

,0,1,2,3,4,5,6,7,8,9,10,11,12
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,PRICES
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Table 1: Construction Material Price Indices,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,United Kingdom,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2015=100
...,...,...,...,...,...,...,...,...,...,...,...,...,...
75,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
76,Notes: 1. These indices are weighted averages ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Source ONS/BEIS
77,electrical or mechanical engineering materials.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
78,2. All series have been rebased to 2015 = 100 ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [66]:
#Data cleaning from meta-data and empty rows and columns
df_9_2 = cleaning_meta_data_empty_rows_columns(df_9_2, 5)
df_9_2

,,2015,2016,2017,2018,2019,2020,
0,(a) New Housing,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,January,100.500000,98.500000,102.800000,107.700000,111.700000,110.900000,NaN
2,February,100.600000,99.100000,103.900000,109.300000,112.400000,111.400000,NaN
3,March,100.600000,98.900000,104.300000,109.300000,113.400000,111.700000,NaN
4,April,100.600000,99.400000,104.800000,109.700000,113.100000,111.700000,NaN
5,May,100.900000,99.800000,105.200000,110.100000,112.800000,112.500000,NaN
6,June,100.500000,100.300000,105.300000,110.500000,113.300000,111.900000,NaN
7,July,100.200000,100.200000,105.400000,111.000000,113.300000,112.200000,NaN
8,August,99.800000,100.800000,105.600000,111.100000,112.800000,112.300000,NaN
9,September,99.400000,101.400000,106.400000,111.600000,112.500000,112.900000,NaN


In [67]:
#Check info of the dataframe
df_9_2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60 entries, 0 to 59
Data columns (total 8 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0           59 non-null     object 
 1   2015    52 non-null     float64
 2   2016    52 non-null     float64
 3   2017    52 non-null     float64
 4   2018    52 non-null     float64
 5   2019    52 non-null     float64
 6   2020    52 non-null     float64
 7           18 non-null     object 
dtypes: float64(6), object(2)
memory usage: 3.9+ KB


In [68]:
#Drop unnecessary rows and columns
df_9_2 = df_9_2.iloc[43:55, :7]
df_9_2

,,2015,2016,2017,2018,2019,2020
43,January,101.2,97.4,102.6,107.1,111.8,110.7
44,February,101.0,97.7,103.3,108.0,112.6,111.0
45,March,101.3,98.1,103.6,108.5,113.1,111.2
46,April,100.7,98.3,103.9,108.7,112.9,113.5
47,May,100.9,99.2,104.0,109.3,112.9,113.9
48,June,100.6,99.5,103.9,109.7,112.3,111.8
49,July,100.6,99.6,104.1,110.1,112.7,111.6
50,August,99.8,99.8,104.4,110.4,112.2,111.6
51,September,99.1,100.2,105.1,111.0,112.3,111.6
52,October,98.7,100.2,106.0,111.0,111.9,113.7


In [69]:
#Coverting data from wide format to long date format:
df_9_2 = reshape_from_wide_to_date_series(df_9_2, 'Construction Material Price Index, 2015 = 100')

df_9_2.head(3)

,"Construction Material Price Index, 2015 = 100",Date
0,101.2,2015-01-01
1,101.0,2015-02-01
2,101.3,2015-03-01


In [70]:
# Determine the required data interval:
print(df_9_2[df_9_2['Date'] == '2015-01-01'])
print(df_9_2[df_9_2['Date'] == '2019-12-01'])

   Construction Material Price Index, 2015 = 100       Date
0                                          101.2 2015-01-01
    Construction Material Price Index, 2015 = 100       Date
59                                          110.1 2019-12-01


In [71]:
#Extracting the required data interval:
df_9_2 = df_9_2.iloc[:60]

print(df_9_2.head(3))

   Construction Material Price Index, 2015 = 100       Date
0                                          101.2 2015-01-01
1                                          101.0 2015-02-01
2                                          101.3 2015-03-01


#### 9_3. Construction Material price indices for 2010 - 2015 years

- Source: Department for Business & Trade Gov.uk
- Frequency: monthly
- Coverage: Jan 2010 - Dec 2015
- Release date: 03-Feb-2016
- Base Index: 2010 = 100

In [72]:
#Extract data from a saved file
df_9_3 = pd.read_csv('df_9_3_raw.csv')
df_9_3.head(10)

,0,1,2,3,4,5,6,7,8,9,10,11,12
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,PRICES
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Table 1: Construction Material Price Indices,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,United Kingdom,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2010=100
5,NaN,2010.0,NaN,2011.0,NaN,2012.0,NaN,2013.0,NaN,2014.0,NaN,2015.0,NaN
6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,(a) New Housing,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,January,96.1,NaN,102.2,NaN,106.0,NaN,107.1,NaN,108.6,NaN,109.8,NaN


In [73]:
#Data cleaning from meta-data and empty rows and columns
df_9_3 = cleaning_meta_data_empty_rows_columns(df_9_3, 5)
df_9_3

,,2010,2011,2012,2013,2014,2015,
0,(a) New Housing,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,January,96.100000,102.200000,106.000000,107.100000,108.600000,109.800000,NaN
2,February,96.500000,103.100000,106.400000,107.600000,109.400000,109.900000,NaN
3,March,96.900000,103.700000,107.600000,107.900000,110.000000,109.900000,NaN
4,April,98.400000,104.200000,107.900000,108.100000,110.100000,109.900000,NaN
5,May,99.900000,104.800000,107.900000,108.500000,110.300000,110.200000,NaN
6,June,100.800000,105.700000,107.300000,108.200000,110.100000,109.700000,NaN
7,July,101.300000,106.000000,107.300000,108.000000,110.500000,109.400000,NaN
8,August,101.700000,106.600000,107.200000,108.200000,110.400000,109.000000,NaN
9,September,102.000000,106.500000,107.100000,108.100000,110.300000,108.500000,NaN


In [74]:
#Drop unnecessary rows and columns
df_9_3 = df_9_3.iloc[43:55, :7]
df_9_3.head(3)

,,2010,2011,2012,2013,2014,2015
43,January,95.7,103.1,109.0,110.0,110.4,110.2
44,February,96.2,104.4,109.3,110.2,111.3,110.0
45,March,96.7,105.6,110.4,110.8,111.5,110.3


In [75]:
#Converting data from wide format to long date format:
df_9_3 = reshape_from_wide_to_date_series(df_9_3, 'Construction Material Price Index, 2010 = 100')
df_9_3.head(3)

,"Construction Material Price Index, 2010 = 100",Date
0,95.7,2010-01-01
1,96.2,2010-02-01
2,96.7,2010-03-01


In [76]:
#Reassign base index from 2010 = 100 to 2015 = 100:
#Find ratio: (annual index for 2015-01-01 from df_9.2) / (annual index for 2015-01-01 from df_9.3) = 101.2/110.2 = 0.9183
#Multiply all df_3 by 0.9183:

df_9_3['Construction Material Price Index, 2015 = 100'] = df_9_3['Construction Material Price Index, 2010 = 100'] * 0.9183
df_9_3 = df_9_3.drop(columns = ['Construction Material Price Index, 2010 = 100'])

df_9_3.head(3)

,Date,"Construction Material Price Index, 2015 = 100"
0,2010-01-01,87.88131
1,2010-02-01,88.34046
2,2010-03-01,88.79961


In [77]:
# Determine the required data interval:
print(df_9_3[df_9_3['Date'] == '2010-01-01'])
print(df_9_3[df_9_3['Date'] == '2014-12-01'])

        Date  Construction Material Price Index, 2015 = 100
0 2010-01-01                                       87.88131
         Date  Construction Material Price Index, 2015 = 100
59 2014-12-01                                      101.28849


In [78]:
#Extracting the required data interval:
df_9_3 = df_9_3.iloc[:60]

df_9_3.tail(3)

,Date,"Construction Material Price Index, 2015 = 100"
57,2014-10-01,101.74764
58,2014-11-01,101.56398
59,2014-12-01,101.28849


#### 9_4. Construction Material price indices for 2006 - 2011 years

- Source: Department for Business & Trade Gov.uk
- Frequency: monthly
- Coverage: Jan 2006 - Dec 2011
- Release date: 01-Feb-2012
- Base Index: 2005 = 100

In [79]:
#Extract data from a saved file
df_9_4 = pd.read_csv('df_9_4_raw.csv')
df_9_4.head(10)

,0,1,2,3,4,5,6,7,8,9,10,11,12
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,PRICES
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Table 1: Construction Material Price Indices,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,United Kingdom,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2005 = 100,NaN
5,NaN,2006.0,NaN,2007.0,NaN,2008.0,NaN,2009.0,NaN,2010.0,NaN,2011,NaN
6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,(a) New Housing,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,January,101.6,NaN,109.8,NaN,115.3,NaN,119.7,NaN,122.3,NaN,130.1,NaN


In [80]:
#Data cleaning from meta-data and empty rows and columns
df_9_4 = cleaning_meta_data_empty_rows_columns(df_9_4, 5)
df_9_4

,,2006,2007,2008,2009,2010,2011,
0,(a) New Housing,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,January,101.600000,109.800000,115.300000,119.700000,122.300000,130.1,NaN
2,February,102.600000,111.700000,116.800000,120.600000,122.900000,131.2,NaN
3,March,103.400000,112.400000,117.500000,120.600000,123.400000,132,NaN
4,April,103.600000,112.300000,118.200000,120.200000,125.200000,132.6,NaN
5,May,104.600000,113.700000,119.000000,119.800000,127.200000,133.4,NaN
6,June,105.800000,114.400000,118.700000,119.500000,128.300000,134.6,NaN
7,July,106.500000,115.200000,120.000000,119.100000,129.000000,135,NaN
8,August,107.600000,116.300000,120.500000,120.200000,129.500000,135.7,NaN
9,September,107.700000,116.300000,120.400000,121.000000,129.900000,135.575563038613,NaN


In [81]:
#Drop unnecessary rows and columns
df_9_4 = df_9_4.iloc[43:55, :7]
df_9_4.head(3)

,,2006,2007,2008,2009,2010,2011
43,January,101.2,110.6,115.3,122.5,124.6,134.2
44,February,102.1,112.3,116.9,122.0,125.2,135.9
45,March,103.1,112.8,117.9,121.6,125.9,137.4


In [82]:
#Reshaping data from wide format to long date format:
df_9_4 = reshape_from_wide_to_date_series(df_9_4, 'Construction Material Price Index, 2005 = 100')
df_9_4.head(3)

,"Construction Material Price Index, 2005 = 100",Date
0,101.2,2006-01-01
1,102.1,2006-02-01
2,103.1,2006-03-01


In [83]:
#Converting the column ''Construction Material Price Index, 2005 = 100' type from object to float
df_9_4['Construction Material Price Index, 2005 = 100'] = df_9_4['Construction Material Price Index, 2005 = 100'].astype('float')
df_9_4.dtypes

Construction Material Price Index, 2005 = 100           float64
Date                                             datetime64[ns]
dtype: object

In [84]:
#Add missing data for 2005 year with value 100, then  reassign base index from 2005 = 100 to 2015 = 100 

#df for missing 2005 year:
#Create the 'Date' column with monthly frequency
dates = pd.date_range(start='2005-01-01', end='2005-12-01', freq='MS')

#Create the 'Const material PI, 2005 = 100' column with float values of 100
values = [100.0] * len(dates)

#Create the DataFrame for 2005 year
df_2005 = pd.DataFrame({'Construction Material Price Index, 2005 = 100': values,'Date': dates})

#Merging df_9_4 and df_2005
df_9_4 = pd.concat([df_9_4, df_2005], ignore_index=True).sort_values(by='Date').reset_index(drop=True)

#Find ratio: annual index for 2010-01-01 from df_9_3 / annual index for 2010-01-01 from df_9_4 = 87.881/124.6 = 0.7053
#Multiply all df_9_4 by 0.7053:
df_9_4['Construction Material Price Index, 2015 = 100'] = df_9_4['Construction Material Price Index, 2005 = 100'] * 0.7053
df_9_4 = df_9_4.drop(columns = ['Construction Material Price Index, 2005 = 100'])


df_9_4

,Date,"Construction Material Price Index, 2015 = 100"
0,2005-01-01,70.530000
1,2005-02-01,70.530000
2,2005-03-01,70.530000
3,2005-04-01,70.530000
4,2005-05-01,70.530000
...,...,...
79,2011-08-01,100.082070
80,2011-09-01,99.957843
81,2011-10-01,99.588360
82,2011-11-01,99.658890


In [85]:
# Determine the required data interval:
print(df_9_4[df_9_4['Date'] == '2005-01-01'])
print(df_9_4[df_9_4['Date'] == '2009-12-01'])

        Date  Construction Material Price Index, 2015 = 100
0 2005-01-01                                          70.53
         Date  Construction Material Price Index, 2015 = 100
59 2009-12-01                                       87.95091


In [86]:
#Extracting the required data interval:
df_9_4 = df_9_4.iloc[0:60]

df_9_4.tail(3)

,Date,"Construction Material Price Index, 2015 = 100"
57,2009-10-01,87.03402
58,2009-11-01,88.09197
59,2009-12-01,87.95091


#### Merging all time serieses for construction material price indicator from Jan 2005 till Jan 2025

In [87]:
# Merge df_9_1, df_9_2, df_9_3, df_9_4:
df_9 = pd.concat([df_9_1, df_9_2, df_9_3, df_9_4], ignore_index=True).sort_values(by='Date').reset_index(drop=True)
df_9['Construction Material Price Index, 2015 = 100'] = df_9['Construction Material Price Index, 2015 = 100'].astype('float').round(2)

#Setting the column 'Date' as an index
df_9 = df_9.set_index('Date')

print(df_9.dtypes)
df_9

Construction Material Price Index, 2015 = 100    float64
dtype: object


,"Construction Material Price Index, 2015 = 100"
Date,
2005-01-01,70.53
2005-02-01,70.53
2005-03-01,70.53
2005-04-01,70.53
2005-05-01,70.53
...,...
2024-09-01,153.00
2024-10-01,152.20
2024-11-01,152.90


#### Indicator 10: Number of construction firms

- Source: Office for National Statistics (ONS)
- Frequency: annually
- Coverage: 1997 - 2023
- Release date: 22-Nov-2024
- Next release: unknown

In [88]:
#Extract data from a saved file
df_10 = pd.read_csv('df_10_raw.csv')
df_10.head(10)

,0,1,2,3,4,5,6,7,8,9,...,18,19,20,21,22,23,24,25,26,27
0,Table 3.1: Construction firms: Number of Firm...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,This worksheet contains one table.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,This table refers to [note 16][note 17][note 1...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,"Some shorthand is used in this table, [x] = un...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Source: Office for National Statistics and Int...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,Number of Business' classified to Construction...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,Size of Firm,1997,1998,1999,2000,2001,2002,2003,2004,2005,...,2014.0,2015.0,2016.0,2017.0,2018.0,2019.0,2020.0,2021.0,2022.0,2023.0
7,0 (sole proprietors),[x],[x],[x],[x],[x],[x],[x],[x],[x],...,40811.0,38711.0,39484.0,39819.0,38888.0,38520.0,36891.0,35352.0,45101.0,37451.0
8,1,86269,87837,88018,87712,77926,71431,70370,71620,73117,...,92926.0,108691.0,123489.0,133472.0,138069.0,144229.0,148584.0,153622.0,157012.0,156153.0
9,2-3,47644,47918,49350,48773,50653,50306,53022,55027,57320,...,66135.0,72128.0,76845.0,82783.0,88297.0,91843.0,94651.0,100218.0,103938.0,103337.0


In [89]:
#Data cleaning from meta-data and empty rows and columns
df_10 = cleaning_meta_data_empty_rows_columns(df_10, 6)
df_10

,Size of Firm,1997,1998,1999,2000,2001,2002,2003,2004,2005,...,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023
0,0 (sole proprietors),[x],[x],[x],[x],[x],[x],[x],[x],[x],...,40811.0,38711.0,39484.0,39819.0,38888.0,38520.0,36891.0,35352.0,45101.0,37451.0
1,1,86269,87837,88018,87712,77926,71431,70370,71620,73117,...,92926.0,108691.0,123489.0,133472.0,138069.0,144229.0,148584.0,153622.0,157012.0,156153.0
2,2-3,47644,47918,49350,48773,50653,50306,53022,55027,57320,...,66135.0,72128.0,76845.0,82783.0,88297.0,91843.0,94651.0,100218.0,103938.0,103337.0
3,4-7,15737,16391,16969,16584,22455,23963,25704,26865,28435,...,29142.0,30855.0,32339.0,33933.0,35434.0,36071.0,36725.0,38353.0,41044.0,40345.0
4,8-13,3787,3988,4148,3790,8044,9819,10508,10982,11599,...,11455.0,11923.0,12255.0,12665.0,12890.0,12908.0,13269.0,13509.0,14378.0,14283.0
5,14-24,3101,3274,3271,3104,4920,5427,5892,6161,6341,...,6016.0,6203.0,6230.0,6328.0,6444.0,6508.0,6633.0,6648.0,7089.0,7291.0
6,25-34,1176,1201,1332,1201,1782,1809,1932,1985,2037,...,1756.0,1722.0,1842.0,1885.0,1939.0,1905.0,1937.0,1993.0,2068.0,1952.0
7,35-59,1156,1263,1188,1109,999,1782,1821,1906,1928,...,1752.0,1849.0,1848.0,1879.0,1932.0,1918.0,1913.0,1847.0,1860.0,1888.0
8,60-79,396,419,397,364,354,457,583,550,573,...,521.0,511.0,534.0,542.0,556.0,549.0,584.0,602.0,617.0,591.0
9,80-114,296,319,304,271,304,425,451,464,469,...,405.0,436.0,468.0,486.0,474.0,445.0,441.0,441.0,433.0,428.0


In [90]:
#Check info of the dataframe
df_10.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15 entries, 0 to 14
Data columns (total 28 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Size of Firm   15 non-null     object 
 1   1997           15 non-null     object 
 2   1998           15 non-null     object 
 3   1999           15 non-null     object 
 4   2000           15 non-null     object 
 5   2001           15 non-null     object 
 6   2002           15 non-null     object 
 7   2003           15 non-null     object 
 8   2004           15 non-null     object 
 9   2005           15 non-null     object 
 10  2006           15 non-null     object 
 11  2007           15 non-null     object 
 12  2008           15 non-null     object 
 13  2009           15 non-null     object 
 14  2010           15 non-null     float64
 15  2011           15 non-null     float64
 16  2012           15 non-null     float64
 17  2013           15 non-null     float64
 18  2014        

In [91]:
# Exclusion of sole proprietors from consideration.
df_10 = df_10.iloc[1:]
df_10.head(3)

,Size of Firm,1997,1998,1999,2000,2001,2002,2003,2004,2005,...,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023
1,1,86269,87837,88018,87712,77926,71431,70370,71620,73117,...,92926.0,108691.0,123489.0,133472.0,138069.0,144229.0,148584.0,153622.0,157012.0,156153.0
2,2-3,47644,47918,49350,48773,50653,50306,53022,55027,57320,...,66135.0,72128.0,76845.0,82783.0,88297.0,91843.0,94651.0,100218.0,103938.0,103337.0
3,4-7,15737,16391,16969,16584,22455,23963,25704,26865,28435,...,29142.0,30855.0,32339.0,33933.0,35434.0,36071.0,36725.0,38353.0,41044.0,40345.0


In [92]:
#Converting values from object type to float64 type
df_10.iloc[:, 1:14] = df_10.iloc[:, 1:14].astype(float)

#Transpose df
df_10 = df_10.transpose()
df_10 = df_10.reset_index()

df_10.head(5)

,index,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,Size of Firm,1,2-3,4-7,8-13,14-24,25-34,35-59,60-79,80-114,115-299,300-599,"600-1,199","1,200 and over",All firms
1,1997,86269.0,47644.0,15737.0,3787.0,3101.0,1176.0,1156.0,396.0,296.0,381.0,107.0,60.0,38.0,160148.0
2,1998,87837.0,47918.0,16391.0,3988.0,3274.0,1201.0,1263.0,419.0,319.0,405.0,125.0,56.0,40.0,163236.0
3,1999,88018.0,49350.0,16969.0,4148.0,3271.0,1332.0,1188.0,397.0,304.0,379.0,105.0,58.0,42.0,165561.0
4,2000,87712.0,48773.0,16584.0,3790.0,3104.0,1201.0,1109.0,364.0,271.0,341.0,91.0,51.0,35.0,163426.0


In [93]:
# Header correction
columns = ['Date'] + list(df_10.iloc[0, 1:])
df_10.columns = columns
df_10 = df_10.iloc[1:]

df_10.head(5)

,Date,1,2-3,4-7,8-13,14-24,25-34,35-59,60-79,80-114,115-299,300-599,"600-1,199","1,200 and over",All firms
1,1997,86269.0,47644.0,15737.0,3787.0,3101.0,1176.0,1156.0,396.0,296.0,381.0,107.0,60.0,38.0,160148.0
2,1998,87837.0,47918.0,16391.0,3988.0,3274.0,1201.0,1263.0,419.0,319.0,405.0,125.0,56.0,40.0,163236.0
3,1999,88018.0,49350.0,16969.0,4148.0,3271.0,1332.0,1188.0,397.0,304.0,379.0,105.0,58.0,42.0,165561.0
4,2000,87712.0,48773.0,16584.0,3790.0,3104.0,1201.0,1109.0,364.0,271.0,341.0,91.0,51.0,35.0,163426.0
5,2001,77926.0,50653.0,22455.0,8044.0,4920.0,1782.0,999.0,354.0,304.0,433.0,129.0,68.0,56.0,168123.0


In [94]:
# Grouping of companies into small (up to 59 employees) , medium (60 - 299 employees) and large (300 and more employees).
small_companies = ['1', '2-3', '4-7', '8-13', '14-24', '25-34']
medium_companies = ['35-59', '60-79', '80-114', '115-299']
large_companies = ['300-599', '600-1,199', '1,200 and over']

#to avoid SettingWithCopyWarning create a copy of df_10:
df_10_1 = df_10.copy()
df_10_1['Small construction companies'] = df_10[small_companies].sum(axis = 1)
df_10_1['Medium construction companies'] = df_10[medium_companies].sum(axis = 1)
df_10_1['Large construction companies'] = df_10[large_companies].sum(axis = 1)
df_10_1['Number of all construction companies'] = df_10[(small_companies + medium_companies + large_companies)].sum(axis = 1)


df_10_1 = df_10_1.drop(columns=(small_companies + medium_companies + large_companies + ['All firms']))
df_10 = df_10_1.copy()
df_10.head(3)

,Date,Small construction companies,Medium construction companies,Large construction companies,Number of all construction companies
1,1997,157714.0,2229.0,205.0,160148.0
2,1998,160609.0,2406.0,221.0,163236.0
3,1999,163088.0,2268.0,205.0,165561.0


In [95]:
# Converting the Date column to datetime64 format and creating data with a monthly frequency
df_10['Date'] = pd.to_datetime(df_10['Date'], format='%Y')
df_10 = df_10.set_index('Date')
df_10 = df_10.resample('MS').ffill().reset_index()

df_10.head(5)

,Date,Small construction companies,Medium construction companies,Large construction companies,Number of all construction companies
0,1997-01-01,157714.0,2229.0,205.0,160148.0
1,1997-02-01,157714.0,2229.0,205.0,160148.0
2,1997-03-01,157714.0,2229.0,205.0,160148.0
3,1997-04-01,157714.0,2229.0,205.0,160148.0
4,1997-05-01,157714.0,2229.0,205.0,160148.0


In [96]:
# Determine the required data interval:
print(df_10[df_10['Date'] == '2005-01-01'])

         Date Small construction companies Medium construction companies  \
96 2005-01-01                     178849.0                        3526.0   

   Large construction companies Number of all construction companies  
96                        269.0                             182644.0  


In [97]:
#Extracting the required data interval:
df_10 = df_10.loc[96:]
df_10.head(5)

,Date,Small construction companies,Medium construction companies,Large construction companies,Number of all construction companies
96,2005-01-01,178849.0,3526.0,269.0,182644.0
97,2005-02-01,178849.0,3526.0,269.0,182644.0
98,2005-03-01,178849.0,3526.0,269.0,182644.0
99,2005-04-01,178849.0,3526.0,269.0,182644.0
100,2005-05-01,178849.0,3526.0,269.0,182644.0


In [98]:
# Add missing data from 2023-02-01 till 2025-01-01:
# For each month of 2023 we will spread yearly data, 
#for 2024 and 2025 we will assume a reduction in quantity of companies according to IBIS World: for 2024 -0.03% and for 2025 - 3.9%
#Construction Contractors in the UK - Market Research Report (2014-2029) last updated October 2024:

# df_2023:
#Create the 'Date' column with monthly frequency
dates_2023 = pd.date_range(start='2023-02-01', end='2023-12-01', freq='MS')
# Create other columns
values_small_2023 = [323361.0] * len(dates_2023)
values_medium_2023 = [3467.0] * len(dates_2023)
values_large_2023 = [235.0] * len(dates_2023)
values_total_2023 = [323361.0 + 3467.0 + 235.0]  * len(dates_2023)

df_2023 = pd.DataFrame({'Date': dates_2023,
                        'Small construction companies': values_small_2023,
                        'Medium construction companies': values_medium_2023,
                        'Large construction companies': values_large_2023,
                        'Number of all construction companies': values_total_2023
                       })

# df_2024
#Create the 'Date' column with monthly frequency
dates_2024 = pd.date_range(start='2024-01-01', end='2024-12-01', freq='MS')
#Create other columns
values_small_2024 = [323361.0 * 0.9997] * len(dates_2024)
values_medium_2024 = [3467.0 * 0.9997] * len(dates_2024)
values_large_2024 = [235.0 * 0.9997] * len(dates_2024)
values_total_2024 = [(323361.0 + 3467.0 + 235.0) * 0.9997]  * len(dates_2024)

df_2024 = pd.DataFrame({'Date': dates_2024,
                        'Small construction companies': values_small_2024,
                        'Medium construction companies': values_medium_2024,
                        'Large construction companies': values_large_2024,
                        'Number of all construction companies': values_total_2024
                       })

# df_2025
#Create the 'Date' column with monthly frequency
dates_2025 = pd.date_range(start='2025-01-01', end='2025-01-01', freq='MS')
#Create other columns
values_small_2025 = [323361.0 * 0.9997] * len(dates_2025)
values_medium_2025 = [3467.0 * 0.9997] * len(dates_2025)
values_large_2025 = [235.0 * 0.9997] * len(dates_2025)
values_total_2025 = [(323361.0 + 3467.0 + 235.0) * 0.9997]  * len(dates_2025)

df_2025 = pd.DataFrame({'Date': dates_2025,
                        'Small construction companies': values_small_2025,
                        'Medium construction companies': values_medium_2025,
                        'Large construction companies': values_large_2025,
                        'Number of all construction companies': values_total_2025
                       })

#Merging data:

df_10 = pd.concat([df_10, df_2023, df_2024, df_2025], ignore_index=True).sort_values(by='Date').set_index('Date')
df_10 = df_10.astype('float').round(2)
df_10

,Small construction companies,Medium construction companies,Large construction companies,Number of all construction companies
Date,,,,
2005-01-01,178849.00,3526.00,269.00,182644.00
2005-02-01,178849.00,3526.00,269.00,182644.00
2005-03-01,178849.00,3526.00,269.00,182644.00
2005-04-01,178849.00,3526.00,269.00,182644.00
2005-05-01,178849.00,3526.00,269.00,182644.00
...,...,...,...,...
2024-09-01,323263.99,3465.96,234.93,326964.88
2024-10-01,323263.99,3465.96,234.93,326964.88
2024-11-01,323263.99,3465.96,234.93,326964.88


In [99]:
df_10.dtypes

Small construction companies            float64
Medium construction companies           float64
Large construction companies            float64
Number of all construction companies    float64
dtype: object

#### Indicator 11: Number of employees in the construction sector
- Source: Office for National Statistics (ONS)
- Frequency: annually
- Coverage: 1997 - 2023
- Release date: 22-Nov-2024
- Next release: unknown
- Employees by businesses classified to construction  - 3rd quarter each year: thousands

In [100]:
#Extract data from a saved file
df_11 = pd.read_csv('df_11_raw.csv')
df_11.head(10)

,0,1,2,3,4,5,6,7,8,9,...,18,19,20,21,22,23,24,25,26,27
0,Table 3.4: Construction firms: Total Employees...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,This worksheet contains one table.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,This table refers to [note 16][note 18][note 1...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Source: Office for National Statistics and Int...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Employees by businesses classified to Construc...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,Size of Firm,1997.0,1998.0,1999.0,2000.0,2001.0,2002.0,2003.0,2004.0,2005.0,...,2014.0,2015.0,2016.0,2017.0,2018.000,2019.0,2020.0,2021.0,2022.0,2023.0
6,1,77.4,77.4,183.2,185.9,66.5,63.6,82.9,70.0,77.6,...,92.9,108.7,123.5,133.5,138.100,144.2,148.6,153.6,157.0,156.2
7,2-3,145.2,148.3,120.5,142.8,126.4,92.1,99.8,97.4,112.0,...,152.5,163.9,174.2,187.2,199.499,207.1,213.3,225.1,234.3,232.9
8,4-7,77.8,81.2,119.3,116.0,135.0,126.1,133.3,120.3,136.7,...,146.4,154.4,161.4,169.2,176.841,180.2,183.0,190.5,204.3,200.6
9,8-13,39.7,41.6,48.0,45.3,93.7,106.7,112.6,99.9,110.2,...,113.2,116.9,119.9,123.8,125.672,125.5,129.1,131.0,139.2,138.4


In [101]:
#Data cleaning from meta-data and empty rows and columns
df_11 = cleaning_meta_data_empty_rows_columns(df_11, 5)
df_11

,Size of Firm,1997,1998,1999,2000,2001,2002,2003,2004,2005,...,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023
0,1,77.4,77.4,183.2,185.9,66.5,63.6,82.9,70.0,77.6,...,92.9,108.7,123.5,133.5,138.100,144.2,148.6,153.6,157.0,156.2
1,2-3,145.2,148.3,120.5,142.8,126.4,92.1,99.8,97.4,112.0,...,152.5,163.9,174.2,187.2,199.499,207.1,213.3,225.1,234.3,232.9
2,4-7,77.8,81.2,119.3,116.0,135.0,126.1,133.3,120.3,136.7,...,146.4,154.4,161.4,169.2,176.841,180.2,183.0,190.5,204.3,200.6
3,8-13,39.7,41.6,48.0,45.3,93.7,106.7,112.6,99.9,110.2,...,113.2,116.9,119.9,123.8,125.672,125.5,129.1,131.0,139.2,138.4
4,14-24,56.0,59.5,63.2,65.7,97.9,103.6,104.2,99.7,114.5,...,106.2,109.4,109.9,111.5,113.282,114.1,116.3,116.6,124.3,128.7
5,25-34,34.5,35.1,36.0,40.2,58.6,47.7,56.5,52.8,59.9,...,50.8,50.0,53.4,54.7,56.071,55.4,56.4,58.0,60.2,56.7
6,35-59,51.9,56.6,58.1,53.7,62.6,78.8,82.7,99.5,86.5,...,77.1,81.4,81.4,83.2,85.779,85.3,85.0,81.7,82.4,83.3
7,60-79,27.1,28.8,29.3,28.1,28.8,29.0,37.7,36.0,39.9,...,35.6,35.0,36.5,37.1,37.850,37.6,39.9,41.0,42.1,40.5
8,80-114,28.3,30.9,31.7,29.2,32.2,37.6,45.5,43.8,46.5,...,38.0,40.6,43.4,44.9,44.255,41.5,41.5,41.4,40.8,40.2
9,115-299,68.1,71.1,78.5,68.8,80.9,87.6,100.7,93.4,106.1,...,84.6,87.7,89.2,96.5,98.529,101.2,100.8,94.9,96.6,98.4


In [102]:
#Transpose df
df_11 = df_11.transpose()
df_11 = df_11.reset_index()

# Header correction
columns = ['Date'] + list(df_11.iloc[0, 1:])
df_11.columns = columns
df_11 = df_11.iloc[1:]

#Change data type:
df_11.iloc[:, 1:] = df_11.iloc[:, 1:].astype(float)

df_11.head(3)

,Date,1,2-3,4-7,8-13,14-24,25-34,35-59,60-79,80-114,115-299,300-599,"600-1,199","1,200 and over",All firms
1,1997,77.4,145.2,77.8,39.7,56.0,34.5,51.9,27.1,28.3,68.1,45.5,50.8,76.3,778.5
2,1998,77.4,148.3,81.2,41.6,59.5,35.1,56.6,28.8,30.9,71.1,52.1,49.1,82.0,813.6
3,1999,183.2,120.5,119.3,48.0,63.2,36.0,58.1,29.3,31.7,78.5,47.3,51.6,92.3,958.8


In [103]:
# Grouping of companies into small (up to 59 employees) , medium (60 - 299 employees) and large (300 and more employees).
# small_companies = ['1', '2-3', '4-7', '8-13', '14-24', '25-34']
# medium_companies = ['35-59', '60-79', '80-114', '115-299']
# large_companies = ['300-599', '600-1,199', '1,200 and over']

df_11_1 = df_11.copy()
df_11_1['Employees - Small construction companies'] = (df_11[small_companies].sum(axis = 1) * 1000).astype(int)
df_11_1['Employees - Medium construction companies'] = (df_11[medium_companies].sum(axis = 1) * 1000).astype(int)
df_11_1['Employees - Large construction companies'] = (df_11[large_companies].sum(axis = 1) * 1000).astype(int)
df_11_1['Employees - All construction companies'] = (df_11[(small_companies + medium_companies + large_companies)].sum(axis = 1) * 1000).astype(int)

df_11_1 = df_11_1.drop(columns=(small_companies + medium_companies + large_companies + ['All firms']))
df_11 = df_11_1.copy()

df_11.head(3)

,Date,Employees - Small construction companies,Employees - Medium construction companies,Employees - Large construction companies,Employees - All construction companies
1,1997,430599,175399,172600,778599
2,1998,443100,187400,183200,813700
3,1999,570200,197600,191200,959000


In [104]:
# Converting the Date column to datetime64 format and creating data with a monthly frequency
df_11['Date'] = pd.to_datetime(df_11['Date'], format='%Y')
df_11 = df_11.set_index('Date')
df_11 = df_11.resample('MS').ffill().reset_index()

df_11.head(3)

,Date,Employees - Small construction companies,Employees - Medium construction companies,Employees - Large construction companies,Employees - All construction companies
0,1997-01-01,430599,175399,172600,778599
1,1997-02-01,430599,175399,172600,778599
2,1997-03-01,430599,175399,172600,778599


In [105]:
# Determine the required data interval:
df_11[df_11['Date']=='2005-01-01']

,Date,Employees - Small construction companies,Employees - Medium construction companies,Employees - Large construction companies,Employees - All construction companies
96,2005-01-01,610900,279000,278900,1168800


In [106]:
#Extracting the required data interval:
df_11 = df_11.iloc[96:]
df_11.head(3)

,Date,Employees - Small construction companies,Employees - Medium construction companies,Employees - Large construction companies,Employees - All construction companies
96,2005-01-01,610900,279000,278900,1168800
97,2005-02-01,610900,279000,278900,1168800
98,2005-03-01,610900,279000,278900,1168800


In [107]:
# Add missing data from 2023-02-01 till 2025-01-01:
# For each month of 2023 we will spread yearly data, 
#for 2024 and 2025 we will assume a reduction of employment in construction industry according to CITB estimate: for 2024 -0.37% 
#for 2025 Jan we will assume the same value as for 2024
# df_2023:
#Create the 'Date' column with monthly frequency
dates_2023 = pd.date_range(start='2023-02-01', end='2023-12-01', freq='MS')
# Create other columns
values_small_2023 = [913500] * len(dates_2023)
values_medium_2023 = [262400] * len(dates_2023)
values_large_2023 = [263300] * len(dates_2023)
values_total_2023 = [913500 + 262400 + 263300]  * len(dates_2023)

df_2023 = pd.DataFrame({'Date': dates_2023,
                        'Employees - Small construction companies': values_small_2023,
                        'Employees - Medium construction companies': values_medium_2023,
                        'Employees - Large construction companies': values_large_2023,
                        'Employees - All construction companies': values_total_2023
                       })

# df_2024_2025
#Create the 'Date' column with monthly frequency
dates_2024_2025 = pd.date_range(start='2024-01-01', end='2025-01-01', freq='MS')
#Create other columns
values_small_2024_2025 = [913500 * 0.9963] * len(dates_2024_2025)
values_medium_2024_2025 = [262400 * 0.9963] * len(dates_2024_2025)
values_large_2024_2025 = [263300 * 0.9963] * len(dates_2024_2025)
values_total_2024_2025 = [(913500 + 262400 + 263300) * 0.9963]  * len(dates_2024_2025)

df_2024_2025 = pd.DataFrame({'Date': dates_2024_2025,
                        'Employees - Small construction companies': values_small_2024_2025,
                        'Employees - Medium construction companies': values_medium_2024_2025,
                        'Employees - Large construction companies': values_large_2024_2025,
                        'Employees - All construction companies': values_total_2024_2025
                       })


#Merging data:

df_11 = pd.concat([df_11, df_2023, df_2024_2025], ignore_index=True).sort_values(by='Date').set_index('Date')
df_11 = df_11.astype('int').round(-1)
df_11

,Employees - Small construction companies,Employees - Medium construction companies,Employees - Large construction companies,Employees - All construction companies
Date,,,,
2005-01-01,610900,279000,278900,1168800
2005-02-01,610900,279000,278900,1168800
2005-03-01,610900,279000,278900,1168800
2005-04-01,610900,279000,278900,1168800
2005-05-01,610900,279000,278900,1168800
...,...,...,...,...
2024-09-01,910120,261430,262320,1433870
2024-10-01,910120,261430,262320,1433870
2024-11-01,910120,261430,262320,1433870


#### Indicator 12: Number and value of new construction contracts/orders
- Source: Office for National Statistics (ONS)
- Frequency: annually, quarterly
- Coverage: 1964 Q1 - 2025 Q1
- Release date: 15-May-2025
- Next release: 14-Aug-2025
- Seasonally adjusted data
- £ million

In [108]:
#Extract data from a saved file
df_12 = pd.read_csv('df_12_raw.csv')
df_12.head(10)

,0,1,2,3,4,5,6,7,8,9,10,11
0,Table 2: New Orders for construction: volume s...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,This worksheet contains one table. Some cells ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Source: Office for National Statistics and Bar...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,£ million,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Time period,New housing\npublic,New housing\nprivate,All \nnew housing,Other new work\ninfrastructure,Other new work\nexcluding infrastructure,Private\nindustrial,Private\ncommercial,All\nother work,All\nnew work,Period on period growths (%),Period on same period one year ago growths (%)
5,1964,12193,33978,46171,NaN,20213,9681,14381,44275,90446,NaN,NaN
6,1965,13092,31172,44264,NaN,19061,10009,11896,40966,85230,-0.058,NaN
7,1966,13719,25581,39300,NaN,19310,9527,10648,39485,78785,-0.076,NaN
8,1967,17197,30290,47487,NaN,24783,10035,8839,43657,91144,0.157,NaN
9,1968,14992,25488,40480,NaN,22072,10244,9626,41942,82422,-0.096,NaN


In [109]:
#Data cleaning from meta-data and empty rows and columns
df_12 = cleaning_meta_data_empty_rows_columns(df_12, 4)
df_12

,Time period,New housing\npublic,New housing\nprivate,All \nnew housing,Other new work\ninfrastructure,Other new work\nexcluding infrastructure,Private\nindustrial,Private\ncommercial,All\nother work,All\nnew work,Period on period growths (%),Period on same period one year ago growths (%)
0,1964,12193,33978,46171,NaN,20213,9681,14381,44275,90446,NaN,NaN
1,1965,13092,31172,44264,NaN,19061,10009,11896,40966,85230,-0.058,NaN
2,1966,13719,25581,39300,NaN,19310,9527,10648,39485,78785,-0.076,NaN
3,1967,17197,30290,47487,NaN,24783,10035,8839,43657,91144,0.157,NaN
4,1968,14992,25488,40480,NaN,22072,10244,9626,41942,82422,-0.096,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
301,Jan to Mar 2024,213,2538,2751,1660,1694,1141,3290,7785,10536,0.143,-0.029
302,Apr to June 2024,446,2866,3312,2146,1928,1412,3761,9247,12558,0.192,0.283
303,Jul to Sep 2024,261,1808,2069,2185,1329,1008,2961,7483,9553,-0.239,-0.097
304,Oct to Dec 2024,269,2253,2522,1516,1370,815,2977,6678,9200,-0.037,-0.002


In [110]:
# Determine the required data interval:
df_12[df_12['Time period'] == 'Jan to Mar 2005']

,Time period,New housing\npublic,New housing\nprivate,All \nnew housing,Other new work\ninfrastructure,Other new work\nexcluding infrastructure,Private\nindustrial,Private\ncommercial,All\nother work,All\nnew work,Period on period growths (%),Period on same period one year ago growths (%)
225,Jan to Mar 2005,528,3925,4453,1635,2598,1238,5724,11195,15649,-0.013,-0.045


In [111]:
#Extracting the required data interval:
df_12 = df_12.iloc[225:].reset_index(drop=True)
df_12.head(5)

,Time period,New housing\npublic,New housing\nprivate,All \nnew housing,Other new work\ninfrastructure,Other new work\nexcluding infrastructure,Private\nindustrial,Private\ncommercial,All\nother work,All\nnew work,Period on period growths (%),Period on same period one year ago growths (%)
0,Jan to Mar 2005,528,3925,4453,1635,2598,1238,5724,11195,15649,-0.013,-0.045
1,Apr to Jun 2005,590,4335,4925,1658,2718,1645,5581,11602,16525,0.056,0.05
2,Jul to Sep 2005,600,4432,5032,1989,2801,1487,5514,11791,16823,0.018,0.102
3,Oct to Dec 2005,755,3566,4320,1692,2507,1769,6735,12703,17023,0.012,0.074
4,Jan to Mar 2006,769,3863,4632,1010,2526,1776,7466,12778,17411,0.023,0.113


In [112]:
#Dropping unnecessary columns
df_12 = df_12.drop(columns = ['All \nnew housing', 'All\nother work', 'Period on period growths (%)', 'Period on same period one year ago growths (%)'])
df_12.head(3)

,Time period,New housing\npublic,New housing\nprivate,Other new work\ninfrastructure,Other new work\nexcluding infrastructure,Private\nindustrial,Private\ncommercial,All\nnew work
0,Jan to Mar 2005,528,3925,1635,2598,1238,5724,15649
1,Apr to Jun 2005,590,4335,1658,2718,1645,5581,16525
2,Jul to Sep 2005,600,4432,1989,2801,1487,5514,16823


In [113]:
#Setting new column names
columns = ['Date', 'New Contracts - Public Housing, £m', 'New Contracts - Private Housing, £m', 'New Contracts - Infrastructure, £m', 'New contracts - Other, £m', 'New Contracts - Private Industrial, £m', 'New Contracts - Private Commercial, £m', 'New contracts - All Construction, £m']
df_12.columns = columns
df_12.head(3)

,Date,"New Contracts - Public Housing, £m","New Contracts - Private Housing, £m","New Contracts - Infrastructure, £m","New contracts - Other, £m","New Contracts - Private Industrial, £m","New Contracts - Private Commercial, £m","New contracts - All Construction, £m"
0,Jan to Mar 2005,528,3925,1635,2598,1238,5724,15649
1,Apr to Jun 2005,590,4335,1658,2718,1645,5581,16525
2,Jul to Sep 2005,600,4432,1989,2801,1487,5514,16823


In [114]:
df_12.dtypes

Date                                      object
New Contracts - Public Housing, £m        object
New Contracts - Private Housing, £m       object
New Contracts - Infrastructure, £m        object
New contracts - Other, £m                 object
New Contracts - Private Industrial, £m    object
New Contracts - Private Commercial, £m    object
New contracts - All Construction, £m      object
dtype: object

In [115]:
# Convert 'Date' to the first month of the quarter
df_12[['Month', 'Year']] = df_12['Date'].str.extract(r'(\w+)\s+to\s+\w+\s+(\d{4})')  # Extract start month and year
df_12['Date'] = pd.to_datetime(df_12['Year'] + ' ' + df_12['Month'], format='%Y %b')  # Combine and convert

# Drop helper columns
df_12 = df_12.drop(columns=['Month', 'Year'])

df_12.head(3)

,Date,"New Contracts - Public Housing, £m","New Contracts - Private Housing, £m","New Contracts - Infrastructure, £m","New contracts - Other, £m","New Contracts - Private Industrial, £m","New Contracts - Private Commercial, £m","New contracts - All Construction, £m"
0,2005-01-01,528,3925,1635,2598,1238,5724,15649
1,2005-04-01,590,4335,1658,2718,1645,5581,16525
2,2005-07-01,600,4432,1989,2801,1487,5514,16823


In [116]:
#Converting quarterly data to monthly
df_12 = quarter_to_month_freq_for_list_of_columns(df_12, 'Date', ['New Contracts - Public Housing, £m', 'New Contracts - Private Housing, £m', 'New Contracts - Infrastructure, £m', 'New contracts - Other, £m', 'New Contracts - Private Industrial, £m', 'New Contracts - Private Commercial, £m', 'New contracts - All Construction, £m'])
df_12 = df_12.set_index('Date')
df_12.head(5)

,"New Contracts - Public Housing, £m","New Contracts - Private Housing, £m","New Contracts - Infrastructure, £m","New contracts - Other, £m","New Contracts - Private Industrial, £m","New Contracts - Private Commercial, £m","New contracts - All Construction, £m"
Date,,,,,,,
2005-01-01,176.00,1308.33,545.00,866.0,412.67,1908.00,5216.33
2005-02-01,176.00,1308.33,545.00,866.0,412.67,1908.00,5216.33
2005-03-01,176.00,1308.33,545.00,866.0,412.67,1908.00,5216.33
2005-04-01,196.67,1445.00,552.67,906.0,548.33,1860.33,5508.33
2005-05-01,196.67,1445.00,552.67,906.0,548.33,1860.33,5508.33


#### Merging All Datasets

In [117]:
# Merging all datasets

df = pd.concat([df_1, df_2, df_3, df_4, df_5, df_6, df_7, df_8, df_9, df_10, df_11, df_12], axis=1, join='outer')
df

,CPIH,"GDP, £m","Employment rate, %","BoE Rate, %",GBP/EUR,GBP/USD,"Business Investment, % change","Govt Expenditure, £m","Construction output, £m","Construction Material Price Index, 2015 = 100",...,Employees - Medium construction companies,Employees - Large construction companies,Employees - All construction companies,"New Contracts - Public Housing, £m","New Contracts - Private Housing, £m","New Contracts - Infrastructure, £m","New contracts - Other, £m","New Contracts - Private Industrial, £m","New Contracts - Private Commercial, £m","New contracts - All Construction, £m"
Date,,,,,,,,,,,,,,,,,,,,,
2005-01-01,78.3,167567.33,73.2,4.7500,1.4331,1.8764,-0.17,48114,14699.33,70.53,...,279000,278900,1168800,176.00,1308.33,545.00,866.00,412.67,1908.00,5216.33
2005-02-01,78.5,167567.33,73.1,4.7500,1.4499,1.8871,-0.17,46543,14699.33,70.53,...,279000,278900,1168800,176.00,1308.33,545.00,866.00,412.67,1908.00,5216.33
2005-03-01,78.8,167567.33,72.9,4.7500,1.4440,1.9078,-0.17,46066,14699.33,70.53,...,279000,278900,1168800,176.00,1308.33,545.00,866.00,412.67,1908.00,5216.33
2005-04-01,79.1,168968.67,72.9,4.7500,1.4652,1.8960,18.53,49028,14592.67,70.53,...,279000,278900,1168800,196.67,1445.00,552.67,906.00,548.33,1860.33,5508.33
2005-05-01,79.4,168968.67,72.9,4.7500,1.4611,1.8538,18.53,42293,14592.67,70.53,...,279000,278900,1168800,196.67,1445.00,552.67,906.00,548.33,1860.33,5508.33
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-09-01,133.5,213891.67,74.9,5.0000,1.1903,1.3217,0.73,106042,17692.00,153.00,...,261430,262320,1433870,87.00,602.67,728.33,443.00,336.00,987.00,3184.33
2024-10-01,134.3,214095.67,74.9,5.0000,1.1972,1.3045,-0.63,108156,17684.00,152.20,...,261430,262320,1433870,89.67,751.00,505.33,456.67,271.67,992.33,3066.67
2024-11-01,134.6,214095.67,75.0,4.7976,1.1998,1.2750,-0.63,102631,17795.00,152.90,...,261430,262320,1433870,89.67,751.00,505.33,456.67,271.67,992.33,3066.67


#### Data Normalization and Standartization

In [118]:
# Select numerical columns
numerical_columns = df.select_dtypes(include=[np.number]).columns

# Create dictionaries to store normalized and standardized columns
normalized_cols = {}
standardized_cols = {}

# Calculate normalized and standardized values for each numerical column
for column in numerical_columns:
    # Min-Max Normalization (0 to 1)
    normalized_cols[column + '_normalized'] = (df[column] - df[column].min()) / (df[column].max() - df[column].min())
    
    # Standardization (mean=0, std=1)
    standardized_cols[column + '_standardized'] = (df[column] - df[column].mean()) / df[column].std()

# Convert dictionaries to DataFrames
df_normalized = pd.DataFrame(normalized_cols, index=df.index)
df_standardized = pd.DataFrame(standardized_cols, index=df.index)

df_normalized.head(3)

,CPIH_normalized,"GDP, £m_normalized","Employment rate, %_normalized","BoE Rate, %_normalized",GBP/EUR_normalized,GBP/USD_normalized,"Business Investment, % change_normalized","Govt Expenditure, £m_normalized","Construction output, £m_normalized","Construction Material Price Index, 2015 = 100_normalized",...,Employees - Medium construction companies_normalized,Employees - Large construction companies_normalized,Employees - All construction companies_normalized,"New Contracts - Public Housing, £m_normalized","New Contracts - Private Housing, £m_normalized","New Contracts - Infrastructure, £m_normalized","New contracts - Other, £m_normalized","New Contracts - Private Industrial, £m_normalized","New Contracts - Private Commercial, £m_normalized","New contracts - All Construction, £m_normalized"
Date,,,,,,,,,,,,,,,,,,,,,
2005-01-01,0.000000,0.126817,0.484375,0.823009,0.822412,0.793805,0.360902,0.079379,0.632729,0.0,...,0.682813,0.478178,0.167488,0.395604,0.844478,0.111526,0.534397,0.467188,0.599912,0.784868
2005-02-01,0.003521,0.126817,0.468750,0.823009,0.862298,0.805195,0.360902,0.057956,0.632729,0.0,...,0.682813,0.478178,0.167488,0.395604,0.844478,0.111526,0.534397,0.467188,0.599912,0.784868
2005-03-01,0.008803,0.126817,0.437500,0.823009,0.848291,0.827230,0.360902,0.051451,0.632729,0.0,...,0.682813,0.478178,0.167488,0.395604,0.844478,0.111526,0.534397,0.467188,0.599912,0.784868


In [119]:
df_standardized.head(3)

,CPIH_standardized,"GDP, £m_standardized","Employment rate, %_standardized","BoE Rate, %_standardized",GBP/EUR_standardized,GBP/USD_standardized,"Business Investment, % change_standardized","Govt Expenditure, £m_standardized","Construction output, £m_standardized","Construction Material Price Index, 2015 = 100_standardized",...,Employees - Medium construction companies_standardized,Employees - Large construction companies_standardized,Employees - All construction companies_standardized,"New Contracts - Public Housing, £m_standardized","New Contracts - Private Housing, £m_standardized","New Contracts - Infrastructure, £m_standardized","New contracts - Other, £m_standardized","New Contracts - Private Industrial, £m_standardized","New Contracts - Private Commercial, £m_standardized","New contracts - All Construction, £m_standardized"
Date,,,,,,,,,,,,,,,,,,,,,
2005-01-01,-1.546569,-1.428656,-0.098753,1.413807,1.737523,1.549964,-0.163093,-1.396736,-0.22841,-1.458986,...,1.007691,0.30017,-1.007863,0.059597,1.219189,-0.70489,0.858313,0.435296,1.193939,1.215999
2005-02-01,-1.532923,-1.428656,-0.154230,1.413807,1.879457,1.595284,-0.163093,-1.489115,-0.22841,-1.458986,...,1.007691,0.30017,-1.007863,0.059597,1.219189,-0.70489,0.858313,0.435296,1.193939,1.215999
2005-03-01,-1.512453,-1.428656,-0.265183,1.413807,1.829611,1.682959,-0.163093,-1.517163,-0.22841,-1.458986,...,1.007691,0.30017,-1.007863,0.059597,1.219189,-0.70489,0.858313,0.435296,1.193939,1.215999


#### Checking for Outliers using Z-Score Method

In [120]:
# Z-Score Method (using standardized columns)
z_threshold = 3  # Common threshold for standardized data
outliers_z = df_standardized.abs() > z_threshold
print("Outliers detected by Z-score method (True indicates outlier):\n")
print(outliers_z.sum())  # Number of outliers per column

# Filter rows with at least one outlier
outlier_rows = df_standardized[outliers_z.any(axis=1)]

# Create a DataFrame to show only columns with actual outliers for each row
outlier_display = pd.DataFrame(index=outlier_rows.index)
for col in df_standardized.columns:
    # Include the column only if it has at least one outlier
    if outliers_z[col].any():
        outlier_display[col] = df_standardized.loc[outlier_rows.index, col].where(outliers_z.loc[outlier_rows.index, col])

# Drop rows where all values are NaN
outlier_display = outlier_display.dropna(axis = 0, how='all')

# Add Date column for context and ensure it's part of the output
outlier_display = outlier_display.reset_index()

print("\nRows with Z-score outliers:")
outlier_display

Outliers detected by Z-score method (True indicates outlier):

CPIH_standardized                                             0
GDP, £m_standardized                                          0
Employment rate, %_standardized                               0
BoE Rate, %_standardized                                      0
GBP/EUR_standardized                                          0
GBP/USD_standardized                                          0
Business Investment, % change_standardized                    6
Govt Expenditure, £m_standardized                             0
Construction output, £m_standardized                          2
Construction Material Price Index, 2015 = 100_standardized    0
Small construction companies_standardized                     0
Medium construction companies_standardized                    0
Large construction companies_standardized                     0
Number of all construction companies_standardized             0
Employees - Small construction companies_

,Date,"Business Investment, % change_standardized","Construction output, £m_standardized","New Contracts - Infrastructure, £m_standardized","New Contracts - Private Commercial, £m_standardized"
0,2005-04-01,6.503056,NaN,NaN,NaN
1,2005-05-01,6.503056,NaN,NaN,NaN
2,2005-06-01,6.503056,NaN,NaN,NaN
3,2005-07-01,-3.927506,NaN,NaN,NaN
4,2005-08-01,-3.927506,NaN,NaN,NaN
5,2005-09-01,-3.927506,NaN,NaN,NaN
6,2006-04-01,NaN,NaN,NaN,3.021525
7,2006-05-01,NaN,NaN,NaN,3.021525
8,2006-06-01,NaN,NaN,NaN,3.021525
9,2017-07-01,NaN,NaN,5.708238,NaN


#### Outliers Analysis

1. **Outliers in the Column 'Business Investment, CVM, SA, in £m, % change':**
- The 2005 Q2 anomaly may stem from a data processing error or one-off adjustment later revised. This is supported by the lack of narrative    in economic news archives about a boom that quarter. No economic sector showed 55% real growth—such a result in GFCF would have made         headlines at the time.
- There’s no macroeconomic event in the UK in Q2 2005 to explain a genuine +55.6% real business investment jump.
- There is no major tax change, budget, or election-related fiscal stimulus large enough to explain it.
- The Q2 spike is followed by a steep drop, consistent with an accounting reclassification or delayed reporting being corrected in Q2
- MG Rover collapsed in April 2005 (Q2) — but that would reduce investment, not increase it.
  The collapse may have led to large one-off write-downs or government interventions. If government spent heavily on industrial support or     asset transfers, it may have spiked investment metrics temporarily.
- London 2012 Olympic Bid Success (6 July 2005): Announced at the start of Q3, this may have boosted business confidence in Q2, anticipating   infrastructure investments. However, actual Olympic-related investments (e.g., construction) ramped up later (2008–2012), so this is         unlikely to explain the 56.1% spike.
- In the note to dataset ONS states:
  “It has been identified that the methodology used to estimate elements of purchased software within GFCF has led to some double counting     from 1997 onwards… it will reduce GFCF across the period by around 1.1% per year.”
  While the average impact is small (-0.02% QoQ), it's possible that 2005 Q2 had a major one-off software-related accounting change that       spiked reported business investment. This is likely an artifact of methodology, not actual economic activity.
  
  **Conclusion:**
  The 2005 outliers as primarily data artifacts due to ONS software double counting with minor influence from economic events (MG Rover        collapse, Olympic bid). For modeling (e.g., VAR, lagged regression),  we will     cap Z-scores at ±3 to retain economic signals while        mitigating data     artifacts:
   

In [121]:
# Cap outliers in the column 'Business Investment, CVM, SA, in £m, % change' 
df_standardized['Business Investment, % change_standardized'] = df_standardized['Business Investment, % change_standardized'].clip(-3, 3)

2. **Outliers in the Column 'Total Construction output, seasonally adjusted, m_standardized'**
   
The construction output in Great Britain during Q2 2020 (April–June) shows significant declines, with All Work at 31,629 £ million, a 36.7% drop from 49,955 £ million in Q2 2019. The largest outlier is Total New Housing, which fell by 49.1% to 6,085 £ million. These declines, driven by the COVID-19 lockdown in April–May 2020, reflect site closures, supply chain disruptions, and labor shortages. Infrastructure work was less affected (-11.8%), due to essential projects like NHS Nightingale Hospitals.

**Conclusion:** The outliers in Q2 2020 construction output, particularly in April–May, are attributed to the COVID-19 lockdown, which caused a 40.1% monthly drop in April and a partial recovery in May. These findings align with ONS reports and industry data, confirming the economic shock’s impact. For modeling, Q2 2020 should be treated as a 'black swan' event to avoid distorting analyses. For VAR or lagged regression models dummy variables will be used to account for COVID-19 impacts.

In [122]:
# Create dummy variable
df['COVID_Lockdown'] = df.index.strftime('%Y-%m').isin(['2020-04', '2020-05']).astype(int)
df_normalized['COVID_Lockdown'] = df_normalized.index.strftime('%Y-%m').isin(['2020-04', '2020-05']).astype(int)
df_standardized['COVID_Lockdown'] = df_standardized.index.strftime('%Y-%m').isin(['2020-04', '2020-05']).astype(int)
print(df[['COVID_Lockdown']].head(5))
print(df[df['COVID_Lockdown'] == 1][['COVID_Lockdown']])

            COVID_Lockdown
Date                      
2005-01-01               0
2005-02-01               0
2005-03-01               0
2005-04-01               0
2005-05-01               0
            COVID_Lockdown
Date                      
2020-04-01               1
2020-05-01               1


3. **Outliers in the column 'Infrastructure new contracts, £ m_standardized'**

The outlier in Q3 2017 (6,614 £ million) likely stems from specific infrastructure project - **High Speed 2 (HS2) Contracts Awarded in Q3 2017**
   
UK Government, “HS2 contracts worth £6.6bn awarded for high-speed railway” (17 July 2017, gov.uk). On July 17, 2017, the UK government awarded £6.6 billion in contracts for Phase 1 of HS2 (London to Birmingham). These          contracts covered civil engineering works (e.g., tunnels, embankments) by joint ventures like Balfour Beatty and Carillion. The              announcement aligns with Q3 2017 (Jul–Sep).
The £6.6 billion in HS2 contracts closely matches the 6,614 £ million in the dataset, explaining the outlier. Infrastructure     new orders spiked due to these large-scale contracts, which were recorded as new orders in Q3 2017.
Alignment: The drop to 3,585 £ million in Q4 2017 (-45.8%) reflects the one-off nature of the HS2 contract awards, as no similar         large contracts were signed in Q4.

**Coclusion:** Unlike the 2005 business investment outliers (software double-counting), this is a real economic event tied to documented contract awards. A dummy variable for infrastructure in Q3 2017 will be added to isolate the HS2 effect:

In [123]:
# Create dummy variable
df['HS2_contracts'] = df.index.strftime('%Y-%m').isin(['2017-07', '2017-08', '2017-09']).astype(int)
df_normalized['HS2_contracts'] = df_normalized.index.strftime('%Y-%m').isin(['2017-07', '2017-08', '2017-09']).astype(int)
df_standardized['HS2_contracts'] = df_standardized.index.strftime('%Y-%m').isin(['2017-07', '2017-08', '2017-09']).astype(int)
print(df_standardized[['HS2_contracts']].head(5))
print(df_standardized[df_standardized['HS2_contracts'] == 1][['HS2_contracts']])

            HS2_contracts
Date                     
2005-01-01              0
2005-02-01              0
2005-03-01              0
2005-04-01              0
2005-05-01              0
            HS2_contracts
Date                     
2017-07-01              1
2017-08-01              1
2017-09-01              1


4. **Outliers in the column 'Private Commercial new contracts, £ m_standardized'**
   
The outlier in Private Commercial new orders (~£3,500 million, standardized) in Q2 2006 (Apr to Jun 2006) is primarily due to:

Economic Boom happened in this period, supported by low interest rates (4.5%). High business investment drove demand for commercial developments (e.g., offices, retail).
In Q2 2006 there were contract awards for large commercial projects, including: Westfield London, Canary Wharf expansions, and Liverpool ONE contributed significantly to the ~£3,500 million total.

**Conclusion:** Unlike 2005 business investment errors, this spike is a real economic event, supported by ONS and industry data. A dummy variable will be added to isolate the Commercial constriction Boom in 2006 effect:

In [124]:
#Create dummy variable
df['Commercial_Construction_Boom_2006'] = df.index.strftime('%Y-%m').isin(['2006-04', '2006-05', '2006-06']).astype(int)
df_normalized['Commercial_Construction_Boom_2006'] = df_normalized.index.strftime('%Y-%m').isin(['2006-04', '2006-05', '2006-06']).astype(int)
df_standardized['Commercial_Construction_Boom_2006'] = df_standardized.index.strftime('%Y-%m').isin(['2006-04', '2006-05', '2006-06']).astype(int)
print(df_standardized[['Commercial_Construction_Boom_2006']].head(5))
print(df_standardized[df_standardized['Commercial_Construction_Boom_2006'] == 1][['Commercial_Construction_Boom_2006']])

            Commercial_Construction_Boom_2006
Date                                         
2005-01-01                                  0
2005-02-01                                  0
2005-03-01                                  0
2005-04-01                                  0
2005-05-01                                  0
            Commercial_Construction_Boom_2006
Date                                         
2006-04-01                                  1
2006-05-01                                  1
2006-06-01                                  1


In [125]:
df_standardized.head(3)

,CPIH_standardized,"GDP, £m_standardized","Employment rate, %_standardized","BoE Rate, %_standardized",GBP/EUR_standardized,GBP/USD_standardized,"Business Investment, % change_standardized","Govt Expenditure, £m_standardized","Construction output, £m_standardized","Construction Material Price Index, 2015 = 100_standardized",...,"New Contracts - Public Housing, £m_standardized","New Contracts - Private Housing, £m_standardized","New Contracts - Infrastructure, £m_standardized","New contracts - Other, £m_standardized","New Contracts - Private Industrial, £m_standardized","New Contracts - Private Commercial, £m_standardized","New contracts - All Construction, £m_standardized",COVID_Lockdown,HS2_contracts,Commercial_Construction_Boom_2006
Date,,,,,,,,,,,,,,,,,,,,,
2005-01-01,-1.546569,-1.428656,-0.098753,1.413807,1.737523,1.549964,-0.163093,-1.396736,-0.22841,-1.458986,...,0.059597,1.219189,-0.70489,0.858313,0.435296,1.193939,1.215999,0,0,0
2005-02-01,-1.532923,-1.428656,-0.154230,1.413807,1.879457,1.595284,-0.163093,-1.489115,-0.22841,-1.458986,...,0.059597,1.219189,-0.70489,0.858313,0.435296,1.193939,1.215999,0,0,0
2005-03-01,-1.512453,-1.428656,-0.265183,1.413807,1.829611,1.682959,-0.163093,-1.517163,-0.22841,-1.458986,...,0.059597,1.219189,-0.70489,0.858313,0.435296,1.193939,1.215999,0,0,0


In [126]:
#Create dummy variable for financial crisis Dec 2007 - Jun 2009
df['financial_crisis_2008'] = ((df.index >= '2007-12-01') & (df.index <= '2009-07-01')).astype(int)
df_normalized['financial_crisis_2008'] = ((df_normalized.index >= '2007-12-01') & (df_normalized.index <= '2009-07-01')).astype(int)
df_standardized['financial_crisis_2008'] = ((df_standardized.index >= '2007-12-01') & (df_standardized.index <= '2009-07-01')).astype(int)

In [127]:
#Create dummy variable for brexit referendum Jun 2016
df['brexit_referendum_2016'] = (df.index == '2016-06-01').astype(int)
df_normalized['brexit_referendum_2016'] = (df_normalized.index == '2016-06-01').astype(int)
df_standardized['brexit_referendum_2016'] = (df_standardized.index == '2016-06-01').astype(int)

In [128]:
#Saving final df as csv:
df.to_csv('df_final.csv', index=True)

In [129]:
#Saving final df as csv:
df_normalized.to_csv('df_normalized_final.csv', index=True)

In [130]:
#Saving final df as csv:
df_standardized.to_csv('df_standardized_final.csv', index=True)

In [131]:
df

,CPIH,"GDP, £m","Employment rate, %","BoE Rate, %",GBP/EUR,GBP/USD,"Business Investment, % change","Govt Expenditure, £m","Construction output, £m","Construction Material Price Index, 2015 = 100",...,"New Contracts - Infrastructure, £m","New contracts - Other, £m","New Contracts - Private Industrial, £m","New Contracts - Private Commercial, £m","New contracts - All Construction, £m",COVID_Lockdown,HS2_contracts,Commercial_Construction_Boom_2006,financial_crisis_2008,brexit_referendum_2016
Date,,,,,,,,,,,,,,,,,,,,,
2005-01-01,78.3,167567.33,73.2,4.7500,1.4331,1.8764,-0.17,48114,14699.33,70.53,...,545.00,866.00,412.67,1908.00,5216.33,0,0,0,0,0
2005-02-01,78.5,167567.33,73.1,4.7500,1.4499,1.8871,-0.17,46543,14699.33,70.53,...,545.00,866.00,412.67,1908.00,5216.33,0,0,0,0,0
2005-03-01,78.8,167567.33,72.9,4.7500,1.4440,1.9078,-0.17,46066,14699.33,70.53,...,545.00,866.00,412.67,1908.00,5216.33,0,0,0,0,0
2005-04-01,79.1,168968.67,72.9,4.7500,1.4652,1.8960,18.53,49028,14592.67,70.53,...,552.67,906.00,548.33,1860.33,5508.33,0,0,0,0,0
2005-05-01,79.4,168968.67,72.9,4.7500,1.4611,1.8538,18.53,42293,14592.67,70.53,...,552.67,906.00,548.33,1860.33,5508.33,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-09-01,133.5,213891.67,74.9,5.0000,1.1903,1.3217,0.73,106042,17692.00,153.00,...,728.33,443.00,336.00,987.00,3184.33,0,0,0,0,0
2024-10-01,134.3,214095.67,74.9,5.0000,1.1972,1.3045,-0.63,108156,17684.00,152.20,...,505.33,456.67,271.67,992.33,3066.67,0,0,0,0,0
2024-11-01,134.6,214095.67,75.0,4.7976,1.1998,1.2750,-0.63,102631,17795.00,152.90,...,505.33,456.67,271.67,992.33,3066.67,0,0,0,0,0


### Conclusion

The preprocessing phase produced a clean dataset of 241 rows and 28 columns (from January 2005 to January 2025), with data aligned to a monthly frequency. Normalized and standardized datasets were created. Outliers were detected using Z-scores. Dummy variables were created for events like COVID-19 and HS2 project announcement. Datasets were saved as df_final.csv, df_normalized_final.csv and df_standardized_final.csv, the datasets are ready for exploratory analysis and modeling to explore macroeconomic-construction relationships.

### Authors

[Alisa Makhonina](https://www.linkedin.com/in/alisa-makhonina-data-science/) Data scientist with over 8 years of experience in construction cost engineering. Structural Engineering graduate.